In [2]:
import json
import pandas as pd
import numpy as np

In [ ]:
all_sheets = pd.read_excel('Raw Data/SCB_act_dbie.xlsx', sheet_name=None)
ms = pd.read_excel('Raw Data/money_stock.xlsx')
cred_deploy = pd.read_excel('Raw Data/cred_deploy.xlsx')
print("Sheet Names:", all_sheets.keys())

/opt/anaconda3/envs/Py_3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Sheet Names: dict_keys(['Major Balance Sheet Items', 'Capital Adequacy', 'Assest Quality', 'Profitability', 'Performance Indicators', 'Major Sectoral', 'Priority-Non_Priority Break-up', 'Infrastructure Financing Items'])


/opt/anaconda3/envs/Py_3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
all_sheets_2 = pd.read_excel('Raw Data/SCB_act_dbie_2.xlsx', sheet_name=None)
cred_deploy_2 = pd.read_excel('Raw Data/cred_deploy_2.xlsx')
print("Sheet Names:", all_sheets_2.keys())

/opt/anaconda3/envs/Py_3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Sheet Names: dict_keys(['Major Balance Sheet Items', 'Capital Adequacy', 'Assest Quality', 'Profitability', 'Performance Indicators', 'Major Sectoral', 'Priority-Non_Priority Break-up', 'Infrastructure Financing Items'])


In [5]:
Foreign_Banks = ["AB BANK LIMITED", "ABU DHABI COMMERCIAL BANK PJSC" ,"AMERICAN EXPRESS BANKING CORP.", "AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED" ,"BANK OF AMERICA , NATIONAL ASSOCIATION" ,"BANK OF BAHRAIN & KUWAIT B.S.C." ,"BANK OF CEYLON", "BANK OF NOVA SCOTIA", "BARCLAYS BANK PLC" ,"BNP PARIBAS" ,"CITIBANK N.A" ,"COMMONWEALTH BANK OF AUSTRALIA", "COOPERATIEVE RABOBANK U.A." ,"CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK", "CREDIT SUISSE AG", "CTBC BANK CO., LTD." ,"DBS BANK INDIA LTD.", "DEUTSCHE BANK AG" ,"FIRSTRAND BANK LTD" ,"HONGKONG AND SHANGHAI BANKING CORPN.LTD." ,"HSBC BANK OMAN S.A.O.G." ,"INDUSTRIAL AND COMMERCIAL BANK OF CHINA", "JPMORGAN CHASE BANK NATIONAL ASSOCIATION", "JSC VTB BANK", "KBC BANK NV" ,"KRUNG THAI BANK PUBLIC COMPANY LIMITED" ,"MASHREQ BANK PSC", "MIZUHO BANK LTD", "MUFG Bank Ltd", "NATIONAL AUSTRALIA BANK", "NatWest Markets Plc", "SBERBANK" ,"SBM BANK (INDIA) LTD." ,"SHINHAN BANK" ,"SOCIETE GENERALE" ,"SONALI BANK", "STANDARD CHARTERED BANK" ,"UBS AG" ,"UNITED OVERSEAS BANK LTD" ,"WOORI BANK", "SUMITOMO MITSUI BANKING CORPORATION", "WESTPAC BANKING CORPORATION", "PT BANK MAYBANK INDONESIA TBK", "DOHA BANK Q.P.S.C", "INDUSTRIAL BANK OF KOREA", "FIRST ABU DHABI BANK PJSC", "KEB HANA BANK", "Qatar National Bank (Q.P.S.C.)", "EMIRATES NBD BANK (P.J.S.C.)", "BANK OF CHINA LIMITED", "KOOKMIN BANK"]
Private_Banks = ["AXIS BANK LIMITED", "CITY UNION BANK LIMITED", "CSB BANK LIMITED", "DCB BANK LIMITED", "FEDERAL BANK LTD", "HDFC BANK LTD.", "ICICI BANK LIMITED", "IDBI BANK LIMITED", "INDUSIND BANK LTD", "ING VYSYA BANK LTD", "JAMMU & KASHMIR BANK LTD", "KARNATAKA BANK LTD" ,"KARUR VYSYA BANK LTD" ,"KOTAK MAHINDRA BANK LTD.", "LAKSHMI VILAS BANK LTD" ,"NAINITAL BANK LTD", "RBL BANK LIMITED", "SOUTH INDIAN BANK LTD", "TAMILNAD MERCANTILE BANK LTD", "THE DHANALAKSHMI BANK LTD" ,"YES BANK LTD.", "IDFC FIRST BANK LIMITED", "BANDHAN BANK LIMITED"]
Public_Banks = ["ALLAHABAD BANK", "ANDHRA BANK", "BANK OF BARODA", "BANK OF INDIA", "BANK OF MAHARASHTRA", "CANARA BANK", "CENTRAL BANK OF INDIA", "CORPORATION BANK", "DENA BANK", "IDBI BANK LIMITED", "INDIAN BANK", "INDIAN OVERSEAS BANK", "ORIENTAL BANK OF COMMERCE", "PUNJAB AND SIND BANK", "PUNJAB NATIONAL BANK", "STATE BANK OF BIKANER AND JAIPUR", "STATE BANK OF HYDERABAD", "STATE BANK OF INDIA", "STATE BANK OF MYSORE", "STATE BANK OF PATIALA", "STATE BANK OF TRAVANCORE", "SYNDICATE BANK", "UCO BANK", "UNION BANK OF INDIA", "UNITED BANK OF INDIA", "VIJAYA BANK", "BHARATIYA MAHILA BANK LTD."]
Small_Finance = ["CAPITAL SMALL FINANCE BANK LIMITED", "EQUITAS SMALL FINANCE BANK LIMITED", "ESAF SMALL FINANCE BANK LIMITED", "FINCARE SMALL FINANCE BANK LIMITED", "SURYODAY SMALL FINANCE BANK LIMITED", "UJJIVAN SMALL FINANCE BANK LIMITED", "AU SMALL FINANCE BANK LIMITED", "NORTH EAST SMALL FINANCE BANK LIMITED", "UTKARSH SMALL FINANCE BANK LIMITED", "JANA SMALL FINANCE BANK LIMITED"]
All_Banks = Foreign_Banks + Private_Banks + Public_Banks + Small_Finance
print(f"{len(Foreign_Banks)}, {len(Private_Banks)} , {len(Public_Banks)}, {len(Small_Finance)}, {len(All_Banks)}")

51, 23 , 27, 10, 111


In [6]:
def clean_dbie(df, x):
    if x==2: # 2 for how the excel sheet column names are formatted, using two rows for one column name
        df_1 = df.iloc[~df.index.isin([0, 1, len(df)-2, len(df) - 1])]
        df_1.pop('Unnamed: 0')
        row_index_to_transform = 0
        row_to_transform = df_1.iloc[row_index_to_transform]
        row_filled = row_to_transform.ffill()
        transformed_row = pd.DataFrame(row_filled).T
        df_2 = pd.concat([transformed_row, df_1.drop(2)], ignore_index=True)   
        new_columns = [f"{df_2.iloc[0, i]}_{df_2.iloc[1, i]}" for i in range(0,df_2.shape[1])]
        df_2.columns = new_columns
        df_3 = df_2.iloc[~df_2.index.isin([0, 1])]
        new_names = ['Quarter', 'Banks']
        df_3.columns = new_names + list(df_3.columns[2:])
        df_3.reset_index(inplace=True)
        df_3 = df_3.drop("index", axis = 1)
        df_3.at[df_3.shape[0] - 1, 'Quarter'] = np.nan
        column_to_transform = 'Quarter'
        col_to_transform = df_3[column_to_transform]
        col_to_transform_filled = col_to_transform.ffill()
        transformed_col = pd.DataFrame(col_to_transform_filled)
        df_4 = pd.concat([transformed_col, df_3.drop(columns=column_to_transform)], axis=1)
        df_4['Quarter'] = df_4['Quarter'].dt.to_period(freq='Q')
        values_to_remove = ['JSC VTB BANK', 'Commercial Banks', 'Foreign Bank Group', 'Private Sector Banks', 'Public Sector Banks', 'Small Finance Bank Group-Scheduled', 'All Scheduled Commercial Banks']
        mask = df_4.isin(values_to_remove).any(axis=1)
        df_5 = df_4[~mask]
        df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
        df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
        df_5['Public_Dummy'] = df_5['Banks'].isin(Public_Banks).astype(int)
        df_5['Small_Finance_Dummy'] = df_5['Banks'].isin(Small_Finance).astype(int)
        df_5.reset_index(inplace=True)
        df_5 = df_5.drop("index", axis = 1)
        return df_5
    elif x == 3: # 3 for how the excel sheet column names are formatted, using two rows for one column name
        df_1 = df.iloc[~df.index.isin([0, 1, 2, len(df)-2, len(df) - 1])]
        df_1.pop('Unnamed: 0')
        row_index_to_transform = 0
        row_to_transform = df_1.iloc[row_index_to_transform]
        row_filled = row_to_transform.ffill()
        transformed_row = pd.DataFrame(row_filled).T
        df_2 = pd.concat([transformed_row, df_1.drop(3)], ignore_index=True)   
        new_columns = [f"{df_2.iloc[0, i]}_{df_2.iloc[1, i]}" for i in range(0,df_2.shape[1])]
        df_2.columns = new_columns
        df_3 = df_2.iloc[~df_2.index.isin([0, 1])]
        new_names = ['Quarter', 'Banks']
        df_3.columns = new_names + list(df_3.columns[2:])
        df_3.reset_index(inplace=True)
        df_3 = df_3.drop("index", axis = 1)
        df_3.at[df_3.shape[0] - 1, 'Quarter'] = np.nan
        column_to_transform = 'Quarter'
        col_to_transform = df_3[column_to_transform]
        col_to_transform_filled = col_to_transform.ffill()
        transformed_col = pd.DataFrame(col_to_transform_filled)
        df_4 = pd.concat([transformed_col, df_3.drop(columns=column_to_transform)], axis=1)
        df_4['Quarter'] = df_4['Quarter'].dt.to_period(freq='Q')
        values_to_remove = ['JSC VTB BANK', 'Commercial Banks', 'Foreign Bank Group', 'Private Sector Banks', 'Public Sector Banks', 'Small Finance Bank Group-Scheduled', 'All Scheduled Commercial Banks']
        mask = df_4.isin(values_to_remove).any(axis=1)
        df_5 = df_4[~mask]
        df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
        df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
        df_5['Public_Dummy'] = df_5['Banks'].isin(Public_Banks).astype(int)
        df_5['Small_Finance_Dummy'] = df_5['Banks'].isin(Small_Finance).astype(int)
        df_5.reset_index(inplace=True)
        df_5 = df_5.drop("index", axis = 1)
        return df_5
    elif x == 'ms': # for money stock data
        df_1 = df.iloc[~df.index.isin([0, 1, 2, 3, len(df)-2, len(df) - 1])]
        df_1.pop('Unnamed: 0')
        row_index_to_transform = 0
        row_to_transform = df_1.iloc[row_index_to_transform]
        row_filled = row_to_transform.ffill()
        transformed_row = pd.DataFrame(row_filled).T
        df_2 = pd.concat([transformed_row, df_1.drop(4)], ignore_index=True)   
        new_columns = [f"{df_2.iloc[0, i]}_{df_2.iloc[1, i]}" for i in range(0,df_2.shape[1])]
        df_2.columns = new_columns
        df_3 = df_2.iloc[~df_2.index.isin([0, 1])]
        new_names = ['Date', 'Banks']
        df_3.columns = new_names + list(df_3.columns[2:])
        df_3.reset_index(inplace=True)
        df_3 = df_3.drop("index", axis = 1)
        return df_3
    else: 
        return print("Error wrong x")

In [7]:
balance_sheet = clean_dbie(all_sheets['Major Balance Sheet Items'], 2)
capital_adequacy = clean_dbie(all_sheets['Capital Adequacy'], 2)
asset_quality = clean_dbie(all_sheets['Assest Quality'], 2)
profitability = clean_dbie(all_sheets['Profitability'], 3)
performance_indicators = clean_dbie(all_sheets['Performance Indicators'], 2)
major_sectoral = clean_dbie(all_sheets['Major Sectoral'], 3)
priority_break_up = clean_dbie(all_sheets['Priority-Non_Priority Break-up'], 3)
infrastructure = clean_dbie(all_sheets['Infrastructure Financing Items'], 2)
money_stock = clean_dbie(ms, 'ms')

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_35600/843860718.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_35600/843860718.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_35600/843860718.py:29: SettingWithCopyWarning: 
A value i

In [8]:
balance_sheet_2 = clean_dbie(all_sheets_2['Major Balance Sheet Items'], 2)
capital_adequacy_2 = clean_dbie(all_sheets_2['Capital Adequacy'], 2)
asset_quality_2 = clean_dbie(all_sheets_2['Assest Quality'], 2)
profitability_2 = clean_dbie(all_sheets_2['Profitability'], 3)
performance_indicators_2 = clean_dbie(all_sheets_2['Performance Indicators'], 2)
major_sectoral_2 = clean_dbie(all_sheets_2['Major Sectoral'], 3)
priority_break_up_2 = clean_dbie(all_sheets_2['Priority-Non_Priority Break-up'], 3)
infrastructure_2 = clean_dbie(all_sheets_2['Infrastructure Financing Items'], 2)

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_35600/843860718.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Foreign_Dummy'] = df_5['Banks'].isin(Foreign_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_35600/843860718.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Private_Dummy'] = df_5['Banks'].isin(Private_Banks).astype(int)
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_35600/843860718.py:29: SettingWithCopyWarning: 
A value i

In [9]:
balance_sheet_cov = pd.concat([balance_sheet, balance_sheet_2])
capital_adequacy_cov = pd.concat([capital_adequacy, capital_adequacy_2])
asset_quality_cov = pd.concat([asset_quality, asset_quality_2])
profitability_cov = pd.concat([profitability, profitability_2])
performance_indicators_cov = pd.concat([performance_indicators, performance_indicators_2])
major_sectoral_cov = pd.concat([major_sectoral, major_sectoral_2])
priority_break_up_cov = pd.concat([priority_break_up, priority_break_up_2])
infrastructure_cov = pd.concat([infrastructure, infrastructure_2])

In [10]:
cred_deploy
df_1 = cred_deploy.iloc[~cred_deploy.index.isin([0, 1, 2, 3, 4, 49, 50, 51, 52])]
df_1.pop('Unnamed: 0')
new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
df_1.columns = new_columns
col = df_1.columns
col_2 = col[2:]
col_3 = list(col_2)
col_3.insert(0, 'Serial No')
col_3.insert(1, 'Industry')
col_3
df_1.columns = col_3
df_2 = df_1.iloc[~df_1.index.isin([5])]
df_2.at[47, 'Serial No'] = '2.18.7'
cred_deploy = df_2

In [11]:
df_3 = cred_deploy.T
new_columns = [f"{df_3.iloc[1,i]}({df_3.iloc[0, i]})" for i in range(0,df_3.shape[1])]
df_3.columns = new_columns
df_3 = df_3.iloc[~df_3.index.isin(['Serial No', 'Industry'])]
df_3 = df_3.reset_index()
cred_deploy = df_3

In [12]:
cred_deploy

,index,Industries (2.1 to 2.19)(2 ),Mining & Quarrying (incl. Coal)(2.1 ),Food Processing(2.2 ),Sugar(2.2.1 ),Edible Oils & Vanaspati(2.2.2 ),Tea(2.2.3 ),Others(2.2.4 ),Beverage & Tobacco(2.3 ),Textiles(2.4 ),...,Construction(2.17 ),Infrastructure(2.18 ),Power(2.18.1 ),Telecommunications(2.18.2 ),Roads(2.18.3 ),Airports(2.18.4 ),Ports(2.18.5 ),Railways(2.18.6 ),Other Infrastructure(2.18.7),Other Industries(2.19 )
0,"June, 29 2012",NaN,334850000000,927360000000,297930000000,124530000000,22160000000,482740000000,146490000000,1613790000000,...,497510000000,6484470000000,3419000000000,976180000000,1158570000000,NaN,NaN,NaN,930720000000,1980710000000
1,"September, 21 2012",NaN,322150000000,959480000000,273510000000,130870000000,26530000000,528580000000,152220000000,1633570000000,...,507270000000,6585030000000,3644760000000,868840000000,1228790000000,NaN,NaN,NaN,842640000000,1859300000000
2,"December, 28 2012",NaN,350230000000,1046100000000,265150000000,159660000000,25870000000,595420000000,159550000000,1706810000000,...,490660000000,6980560000000,3864600000000,931200000000,1265110000000,NaN,NaN,NaN,919640000000,1850020000000
3,"March, 22 2013",NaN,346390000000,1173680000000,329760000000,170530000000,25820000000,647570000000,165110000000,1835360000000,...,521660000000,7297210000000,4158490000000,877650000000,1313120000000,NaN,NaN,NaN,947960000000,1809680000000
4,"June, 28 2013",NaN,317220700000,1231164000000,337914100000,182700700000,27309700000,683239500000,156147600000,1837244200000,...,560356300000,7695997100000,4427325800000,889583700000,1411222200000,NaN,NaN,NaN,967865400000,1770645600000
5,"September, 20 2013",NaN,328890000000,1201530000000,317830000000,176150000000,33290000000,674270000000,159000000000,1855500000000,...,565260000000,7931970000000,4591210000000,873620000000,1437020000000,NaN,NaN,NaN,1030120000000,1841650000000
6,"December, 27 2013",NaN,352150000000,1350330000000,296960000000,219200000000,32550000000,801630000000,163640000000,1927910000000,...,577240000000,8037150000000,4683530000000,856260000000,1510660000000,NaN,NaN,NaN,986690000000,1802620000000
7,"March, 21 2014",25164840000000,358450000000,1462540000000,344660000000,212930000000,32410000000,872530000000,182860000000,2022130000000,...,625710000000,8363560000000,4869020000000,882040000000,1578600000000,NaN,NaN,NaN,1033900000000,1880600000000
8,"June, 27 2014",NaN,373540000000,1487320000000,356340000000,207250000000,29000000000,894730000000,182590000000,1979370000000,...,705020000000,8553630000000,5065570000000,827950000000,1619460000000,NaN,NaN,NaN,1040650000000,1784970000000
9,"September, 19 2014",NaN,385080000000,1408610000000,338700000000,176570000000,31060000000,862270000000,192110000000,1933670000000,...,724540000000,8713090000000,5178410000000,851460000000,1639630000000,NaN,NaN,NaN,1043580000000,1752980000000


In [13]:
cred_deploy_2
df_1 = cred_deploy_2.iloc[~cred_deploy_2.index.isin([0,1,2,3,4,50,51,52,53])]
df_1.pop('Unnamed: 0')
new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
df_1.columns = new_columns
col = df_1.columns
col_2 = col[2:]
col_3 = list(col_2)
col_3.insert(0, 'Serial No')
col_3.insert(1, 'Industry')
col_3
df_1.columns = col_3
df_2 = df_1.iloc[~df_1.index.isin([5])]
cred_deploy_2 = df_2
#cred_deploy_2

In [14]:
df_3 = cred_deploy_2.T
new_columns = [f"{df_3.iloc[1,i]}({df_3.iloc[0, i]} )" for i in range(0,df_3.shape[1])]
df_3.columns = new_columns
df_3 = df_3.iloc[~df_3.index.isin(['Serial No', 'Industry'])]
df_3 = df_3.reset_index()
cred_deploy_2 = df_3
cred_deploy_2.pop('nan(nan )')
cols = cred_deploy.columns
cred_deploy_2.columns = cols
cred_deploy_2

,index,Industries (2.1 to 2.19)(2 ),Mining & Quarrying (incl. Coal)(2.1 ),Food Processing(2.2 ),Sugar(2.2.1 ),Edible Oils & Vanaspati(2.2.2 ),Tea(2.2.3 ),Others(2.2.4 ),Beverage & Tobacco(2.3 ),Textiles(2.4 ),...,Construction(2.17 ),Infrastructure(2.18 ),Power(2.18.1 ),Telecommunications(2.18.2 ),Roads(2.18.3 ),Airports(2.18.4 ),Ports(2.18.5 ),Railways(2.18.6 ),Other Infrastructure(2.18.7),Other Industries(2.19 )
0,"March, 27 2020",2946851.25,45406.79,144268.84,27563.45,16521.68,4929.18,95254.53,18025.49,193452.92,...,127837.59,1083656.43,577326.55,147176.4,181530.8,5143.87,12745.18,10884.73,148848.9,223620.5
1,"June, 19 2020",2911362.69,42253.49,149015.3,26488.3,15629.29,4634.19,102263.52,16533.28,190784.86,...,127892.76,1101626.86,586633.82,150688.11,182055.92,4802.74,14078.11,12174.22,151193.94,205511.88
2,"September, 25 2020",2831679.89,40193.13,139617.66,21317.08,16223.47,5123.63,96953.48,16399.04,189635.98,...,128783.5,1059541.12,564767.64,122346.36,198406.07,5453.63,11755.45,13278.85,143533.12,201124.66
3,"December, 18 2020",2822924.36,42295.59,157785.66,27166.33,17239.32,5278.93,108101.08,15952.44,193322.58,...,127239.73,1043335.55,564109.36,104087.63,202368.61,6478.51,10927.88,12370.93,142992.63,201002.13
4,"March, 26 2021",2934688.691812,43498.190103,156556.826727,26490.400381,17998.84563,5059.66,107007.920716,17804.609372,204971.775735,...,119670.341604,1095467.252598,571028.299087,114960.820779,226299.436161,8573.41,10152.8,12471.11,151981.37657,229431.722417
5,"June, 18 2021",2963016.767421,46552.8493,157335.409076,24002.679447,17437.109093,5389.78,110505.840536,17381.309306,213947.095669,...,124144.353364,1115392.355526,574741.017201,117646.35,238663.130724,9724.12,10389.69,12610.97,151617.077601,206514.197401
6,"September, 24 2021",2953548.022651,48985.889006,151709.299469,19156.628098,16977.779758,5710.56,109864.331613,16837.415474,210523.275347,...,122209.534756,1105033.908499,578801.107737,113414.806633,248586.773064,7593.54,11371.69,13004.32,132261.671065,219683.873383
7,"December, 31 2021",3106358.322852,50962.555427,170429.738847,21427.81473,18063.6129,6077.71,124860.601217,17655.44622,224449.750184,...,120142.261055,1148724.801528,600476.84858,109350.417652,255328.148632,7624.29,9453.43,10193.89,156297.776664,249794.36022
8,"March, 25 2022",3235278.907314,51129.797441,176658.02019,26370.365018,18434.094944,5818.129499,126035.43073,18674.63327,231700.755866,...,122454.776842,1205737.164518,615806.43019,130614.777591,273197.384462,6707.98,8897.95,10604.97,159907.672275,256126.893877
9,"June, 17 2022",3248920.985505,50514.627224,179398.103837,24157.393644,18559.856727,6459.828708,130221.024758,17912.841907,226293.301606,...,123010.935846,1222190.250635,624747.780735,132251.103169,281206.062002,6870.16,7964.87,11588.34,157561.934729,251550.947308


In [15]:
cred_deploy_cov = pd.concat([cred_deploy, cred_deploy_2])

In [ ]:
# # Uncomment to write files to csv
# balance_sheet.to_csv('Clean Data/balance_sheet.csv')
# capital_adequacy.to_csv('Clean Data/capital_adequacy.csv')
# asset_quality.to_csv('Clean Data/asset_quality.csv')
# profitability.to_csv('Clean Data/profitability.csv')
# performance_indicators.to_csv('Clean Data/performance_indicators.csv')
# major_sectoral.to_csv('Clean Data/major_sectoral.csv')
# priority_break_up.to_csv('Clean Data/priority_break_up.csv')
# infrastructure.to_csv('Clean Data/infrastructure.csv')
# money_stock.to_csv('Clean Data/money_stock.csv')
# cred_deploy.to_csv('Clean Data/cred_deploy.csv')

In [ ]:
# # Uncomment to write files to csv
# balance_sheet_cov.to_csv('Clean Data/balance_sheet_cov.csv')
# capital_adequacy_cov.to_csv('Clean Data/capital_adequacy_cov.csv')
# asset_quality_cov.to_csv('Clean Data/asset_quality_cov.csv')
# profitability_cov.to_csv('Clean Data/profitability_cov.csv')
# performance_indicators_cov.to_csv('Clean Data/performance_indicators_cov.csv')
# major_sectoral_cov.to_csv('Clean Data/major_sectoral_cov.csv')
# priority_break_up_cov.to_csv('Clean Data/priority_break_up_cov.csv')
# infrastructure_cov.to_csv('/Users/uddhav/Desktop/Thesis/bank_variables_csv/infrastructure_cov.csv')
# cred_deploy_cov.to_csv('/Users/uddhav/Desktop/Thesis/bank_variables_csv/cred_deploy_cov.csv')

In [18]:
with open('Data/banknames.json', 'r') as file_2:
    data_2 = json.load(file_2)

In [19]:
bank_branch_code = pd.DataFrame([data_2])
bank_branch_code = bank_branch_code.T
bank_branch_code = bank_branch_code.reset_index()
bank_branch_code = bank_branch_code.rename(columns = { 'index' : 'Code', 0 : 'bank_name'})
branch_name_list = bank_branch_code['bank_name'].to_list()
branch_name_list_cap = [x.upper() for x in branch_name_list]
branch_name_list_cap

['ABHYUDAYA COOPERATIVE BANK',
 'ABU DHABI COMMERCIAL BANK',
 'AHMEDABAD MERCANTILE COOPERATIVE BANK',
 'AKOLA JANATA COMMERCIAL COOPERATIVE BANK',
 'ALLAHABAD BANK',
 'ALMORA URBAN CO-OPERATIVE BANK LTD',
 'ANDHRA BANK',
 'ANDHRA PRAGATHI GRAMEENA BANK',
 'APNA SAHAKARI BANK LTD',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD',
 'AXIS BANK',
 'BANDHAN BANK LIMITED',
 'PT BANK MAYBANK INDONESIA TBK',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF BARODA',
 'BANK OF CEYLON',
 'BANK OF INDIA',
 'BANK OF MAHARASHTRA',
 'BANK OF TOKYO MITSUBISHI LTD',
 'BARCLAYS BANK',
 'BASSEIN CATHOLIC CO-OP BANK LTD',
 'BHARAT COOPERATIVE BANK MUMBAI LTD',
 'BHARATIYA MAHILA BANK LTD',
 'BNP PARIBAS BANK',
 'CANARA BANK',
 'CAPITAL LOCAL AREA BANK LTD',
 'CATHOLIC SYRIAN BANK',
 'CENTRAL BANK OF INDIA',
 'CHINATRUST COMMERCIAL BANK',
 'CITI BANK',
 'CITIZEN CREDIT COOPERATIVE BANK',
 'CITY UNION BANK LTD',
 'COMMONWEALTH BANK OF AUSTRALIA',
 'CORPORATION BANK',
 'CREDIT AGRICOLE CORPORATE 

In [20]:
bank_branch_code

,Code,bank_name
0,ABHY,Abhyudaya Cooperative Bank
1,ADCB,Abu Dhabi Commercial Bank
2,AMCB,Ahmedabad Mercantile Cooperative Bank
3,AKJB,Akola Janata Commercial Cooperative Bank
4,ALLA,Allahabad Bank
...,...,...
173,VIJB,Vijaya Bank
174,WPAC,Westpac Banking Corporation
175,HVBK,Woori Bank
176,YESB,Yes Bank Ltd


In [20]:
Foreign_Banks_2 = ["AB BANK LTD", "ABU DHABI COMMERCIAL BANK" ,"AMERICAN EXPRESS BANKING CORP", "AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD" ,"BANK OF AMERICA" ,"BANK OF BAHREIN AND KUWAIT" ,"BANK OF CEYLON", "BANK OF NOVA SCOTIA", "BARCLAYS BANK" ,"BNP PARIBAS BANK" ,"CITI BANK" ,"COMMONWEALTH BANK OF AUSTRALIA", "COOPERATIEVE RABOBANK U.A." ,"CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK", "CREDIT SUISSE AG", "CTBC BANK CO., LTD" ,"DBS BANK INDIA LTD", "DEUTSCHE BANK" ,"FIRSTRAND BANK LTD" ,"HONGKONG AND SHANGHAI BANKING CORPN.LTD" ,"HSBC BANK OMAN S.A.O.G." ,"INDUSTRIAL AND COMMERCIAL BANK OF CHINA LTD", "JP MORGAN CHASE BANK NA", "JSC VTB BANK", "KBC BANK NV" ,"KRUNG THAI BANK PUBLIC COMPANY LIMITED" ,"MASHREQ BANK", "MIZUHO CORPORATE BANK LTD", "MUFG BANK LTD", "NATIONAL AUSTRALIA BANK", "NatWest Markets Plc", "SBER BANK" ,"SBM BANK (INDIA) LTD" ,"SHINHAN BANK" ,"SOCIETE GENERALE" ,"SONALI BANK", "STANDARD CHARTERED BANK" ,"UBS AG" ,"UNITED OVERSEAS BANK" ,"WOORI BANK", "SUMITOMO MITSUI BANKING CORPORATION", "WESTPAC BANKING CORPORATION", "PT BANK MAYBANK INDONESIA TBK", "DOHA BANK QSC", "INDUSTRIAL BANK OF KOREA", "FIRST ABU DHABI BANK PJSC", "KEB HANA BANK", "Qatar National Bank (Q.P.S.C.)", "EMIRATES NBD BANK (P.J.S.C.)", "BANK OF CHINA LTD", "KOOKMIN BANK"]
Private_Banks_2 = ["AXIS BANK", "CITY UNION BANK LTD", "CATHOLIC SYRIAN BANK", "DCB BANK LTD", "FEDERAL BANK LTD", "HDFC BANK LTD", "ICICI BANK LTD", "IDBI LTD", "INDUSIND BANK LTD", "ING VYSYA BANK LTD", "JAMMU & KASHMIR BANK", "KARNATAKA BANK LTD" ,"KARUR VYSYA BANK" ,"KOTAK MAHINDRA BANK", "LAXMI VILAS BANK" ,"NAINITAL BANK LTD", "RBL BANK LIMITED", "SOUTH INDIAN BANK", "TAMILNADU MERCANTILE BANK", "DHANLAXMI BANK LTD" ,"YES BANK LTD", "IDFC BANK LTD", "BANDHAN BANK LIMITED"]
Public_Banks_2 = ["ALLAHABAD BANK", "ANDHRA BANK", "BANK OF BARODA", "BANK OF INDIA", "BANK OF MAHARASHTRA", "CANARA BANK", "CENTRAL BANK OF INDIA", "CORPORATION BANK", "DENA BANK", "IDBI LTD", "INDIAN BANK", "INDIAN OVERSEAS BANK", "ORIENTAL BANK OF COMMERCE", "PUNJAB AND SIND BANK", "PUNJAB NATIONAL BANK", "STATE BANK OF BIKANER AND JAIPUR", "STATE BANK OF HYDERABAD", "STATE BANK OF INDIA", "STATE BANK OF MYSORE", "STATE BANK OF PATIALA", "STATE BANK OF TRAVANCORE", "SYNDICATE BANK", "UCO BANK", "UNION BANK OF INDIA", "UNITED BANK OF INDIA", "VIJAYA BANK", "BHARATIYA MAHILA BANK LTD"]
Small_Finance_2= ["CAPITAL SMALL FINANCE BANK LTD", "EQUITAS SMALL FINANCE BANK LIMITED", "ESAF SMALL FINANCE BANK LTD", "FINCARE SMALL FINANCE BANK LTD", "SURYODAY SMALL FINANCE BANK LTD", "UJJIVAN SMALL FINANCE BANK LTD", "AU SMALL FINANCE BANK LTD", "NORTH EAST SMALL FINANCE BANK LTD", "UTKARSH SMALL FINANCE BANK LTD", "JANA SMALL FINANCE BANK LTD"]
All_Banks_2 = Foreign_Banks_2 + Private_Banks_2 + Public_Banks_2 + Small_Finance_2
All_Banks_2 = [x.upper() for x in All_Banks_2]
All_Banks_2

['AB BANK LTD',
 'ABU DHABI COMMERCIAL BANK',
 'AMERICAN EXPRESS BANKING CORP',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF CEYLON',
 'BANK OF NOVA SCOTIA',
 'BARCLAYS BANK',
 'BNP PARIBAS BANK',
 'CITI BANK',
 'COMMONWEALTH BANK OF AUSTRALIA',
 'COOPERATIEVE RABOBANK U.A.',
 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK',
 'CREDIT SUISSE AG',
 'CTBC BANK CO., LTD',
 'DBS BANK INDIA LTD',
 'DEUTSCHE BANK',
 'FIRSTRAND BANK LTD',
 'HONGKONG AND SHANGHAI BANKING CORPN.LTD',
 'HSBC BANK OMAN S.A.O.G.',
 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA LTD',
 'JP MORGAN CHASE BANK NA',
 'JSC VTB BANK',
 'KBC BANK NV',
 'KRUNG THAI BANK PUBLIC COMPANY LIMITED',
 'MASHREQ BANK',
 'MIZUHO CORPORATE BANK LTD',
 'MUFG BANK LTD',
 'NATIONAL AUSTRALIA BANK',
 'NATWEST MARKETS PLC',
 'SBER BANK',
 'SBM BANK (INDIA) LTD',
 'SHINHAN BANK',
 'SOCIETE GENERALE',
 'SONALI BANK',
 'STANDARD CHARTERED BANK',
 'UBS AG',
 'UNITED OVERSEAS BANK',
 'W

In [21]:
inter_1 = []
All_Banks_3 = [x.upper() for x in All_Banks]
for i in All_Banks_3:
    if i in branch_name_list_cap:
        inter_1.append(i)

len(inter_1)

46

In [22]:
inter_2 = []
for i in All_Banks_2:
    if i not in branch_name_list_cap:
        inter_2.append(i)
inter_2

['AB BANK LTD',
 'AMERICAN EXPRESS BANKING CORP',
 'BANK OF NOVA SCOTIA',
 'COOPERATIEVE RABOBANK U.A.',
 'CTBC BANK CO., LTD',
 'DBS BANK INDIA LTD',
 'HONGKONG AND SHANGHAI BANKING CORPN.LTD',
 'HSBC BANK OMAN S.A.O.G.',
 'JSC VTB BANK',
 'KBC BANK NV',
 'KRUNG THAI BANK PUBLIC COMPANY LIMITED',
 'MUFG BANK LTD',
 'NATWEST MARKETS PLC',
 'SBM BANK (INDIA) LTD',
 'SONALI BANK',
 'UBS AG',
 'FIRST ABU DHABI BANK PJSC',
 'QATAR NATIONAL BANK (Q.P.S.C.)',
 'EMIRATES NBD BANK (P.J.S.C.)',
 'BANK OF CHINA LTD',
 'KOOKMIN BANK',
 'ING VYSYA BANK LTD',
 'JAMMU & KASHMIR BANK',
 'NAINITAL BANK LTD',
 'CAPITAL SMALL FINANCE BANK LTD',
 'ESAF SMALL FINANCE BANK LTD',
 'FINCARE SMALL FINANCE BANK LTD',
 'SURYODAY SMALL FINANCE BANK LTD',
 'UJJIVAN SMALL FINANCE BANK LTD',
 'AU SMALL FINANCE BANK LTD',
 'NORTH EAST SMALL FINANCE BANK LTD',
 'UTKARSH SMALL FINANCE BANK LTD',
 'JANA SMALL FINANCE BANK LTD']

In [23]:
inter_3 = []
for i in All_Banks_2:
    if i in branch_name_list_cap:
        inter_3.append(i)

len(inter_3)

78

In [24]:
inter_4 = []
for i in All_Banks_2:
    if i not in branch_name_list_cap:
        inter_4.append(i)
len(inter_4)

33

In [25]:
inter_5 = []
for i in branch_name_list_cap:
    if i in All_Banks_2:
        inter_5.append(i)

len(inter_5)

77

In [26]:
bank_branch_code['bank_name'] = bank_branch_code['bank_name'].str.upper()
bank_branch_code

,Code,bank_name
0,ABHY,ABHYUDAYA COOPERATIVE BANK
1,ADCB,ABU DHABI COMMERCIAL BANK
2,AMCB,AHMEDABAD MERCANTILE COOPERATIVE BANK
3,AKJB,AKOLA JANATA COMMERCIAL COOPERATIVE BANK
4,ALLA,ALLAHABAD BANK
...,...,...
173,VIJB,VIJAYA BANK
174,WPAC,WESTPAC BANKING CORPORATION
175,HVBK,WOORI BANK
176,YESB,YES BANK LTD


In [27]:
bank_branch_code_filter = bank_branch_code[bank_branch_code['bank_name'].isin(inter_3)]
bank_branch_code_filter

,Code,bank_name
1,ADCB,ABU DHABI COMMERCIAL BANK
4,ALLA,ALLAHABAD BANK
6,ANDB,ANDHRA BANK
9,ANZB,AUSTRALIA AND NEW ZEALAND BANKING GROUP LTD
10,UTIB,AXIS BANK
...,...,...
171,UOVB,UNITED OVERSEAS BANK
173,VIJB,VIJAYA BANK
174,WPAC,WESTPAC BANKING CORPORATION
175,HVBK,WOORI BANK


In [28]:
bank_branch_code_filter = bank_branch_code_filter[bank_branch_code_filter['Code'] != 'ESFB'] #Not in branch locations

In [29]:
banks_to_read = bank_branch_code_filter['Code']
banks_to_read

1      ADCB
4      ALLA
6      ANDB
9      ANZB
10     UTIB
       ... 
171    UOVB
173    VIJB
174    WPAC
175    HVBK
176    YESB
Name: Code, Length: 76, dtype: object

In [30]:
data_dict = {}
for i in banks_to_read:
    with open(f'/Users/uddhav/Desktop/Thesis/by-bank/{i}.json', 'r') as file:
        data_dict[i] = json.load(file)

In [31]:
len(data_dict)

76

In [32]:
for i in banks_to_read:
    globals()[f'bank_branch_{i}'] = pd.json_normalize(data_dict[i].values())

In [33]:
bank_branch_HDFC

,BANK,IFSC,BRANCH,ADDRESS,CONTACT,CITY,DISTRICT,STATE
0,HDFC BANK,HDFC0000001,TULSIANI CHMBRS - NARIMAN PT,101-104 TULSIANI CHAMBERSFREE PRESS JOURNAL MA...,9890603333.0,MUMBAI,GREATER MUMBAI,MAHARASHTRA
1,HDFC BANK,HDFC0000002,MUMBAI - KHAR (WEST),"SWAGATAM,OPP. KHAR POLICE STATION,S.V.ROAD, KH...",9890603333.0,MUMBAI,GREATER MUMBAI,MAHARASHTRA
2,HDFC BANK,HDFC0000003,K G MARG,"209 - 214, KAILASH BUILDING, 26, KASTURBA GAND...",61606161,DELHI,NEW DELHI,DELHI
3,HDFC BANK,HDFC0000004,CHENNAI - ITC CENTRE - ANNA SALAI,"759, ITC CENTREANNA SALAI, OPP T.V.S.CHENNAITA...",9840673333.0,CHENNAI,CHENNAI,TAMIL NADU
4,HDFC BANK,HDFC0000005,MUMBAI - BHULABHAI DESAI ROAD,"TIRUPATI APARTMENTS,SHOP NO. 4,BHULABHAI DESAI...",9890603333.0,MUMBAI,GREATER MUMBAI,MAHARASHTRA
...,...,...,...,...,...,...,...,...
5476,HDFC BANK,HDFC0CYACBP,THE YASHWANT CO-OP BANK LTD PHALTAN,"69,GURUWAR PETH,CHAWDI CHOWK,KARAD 415110",9422745818.0,KARAD,SATARA,MAHARASHTRA
5477,HDFC BANK,HDFC0CYDCBL,YOUTH DEVELOPMENT COOP BANK LTD.KOL,"POORNIMA F-2, FIRST FLOOR, 1429/G, NEAR SAMBHA...",2550120.0,KOLHAPUR,KOLHAPUR,MAHARASHTRA
5478,HDFC BANK,HDFC0CYMCBK,YEOLA MER CO O PBANK LTD,YASHODHAN BALAJI GALLI YEOLA DIST NASIK,265547.0,YEOLA,NASIK,MAHARASHTRA
5479,HDFC BANK,HDFC0CYMSB1,THE YAVATMAL MAHILA SAH BANK LTD,"ANNAPURNA, MAINDE CHOWK, SANKAT MOCHAN ROAD YA...",250661.0,YAVATMAL,YAVATMAL,MAHARASHTRA


In [34]:
branch_all = pd.concat([globals()[f'bank_branch_{i}'] for i in banks_to_read], ignore_index=True)
branch_all['foreign_dummy'] = branch_all['BANK'].isin(Foreign_Banks_2).astype(int)
branch_all['private_dummy'] = branch_all['BANK'].isin(Private_Banks_2).astype(int)
branch_all['public_dummy'] = branch_all['BANK'].isin(Public_Banks_2).astype(int)
branch_all['small_finance_dummy'] = branch_all['BANK'].isin(Small_Finance_2).astype(int)

In [35]:
branch_all

,BANK,IFSC,BRANCH,ADDRESS,CONTACT,CITY,DISTRICT,STATE,foreign_dummy,private_dummy,public_dummy,small_finance_dummy
0,ABU DHABI COMMERCIAL BANK,ADCB0000001,RTGS-HO,"75, REHMAT MANZIL, V. N. ROAD, CURCHGATE, MUMB...",39534100,MUMBAI,MUMBAI CITY,MAHARASHTRA,1,0,0,0
1,ABU DHABI COMMERCIAL BANK,ADCB0000002,BANGALORE,"CITI CENTRE, 28, CHURCH STREET, OFF M. G. ROAD...",25582000,BANGALORE,BANGALORE URBAN,KARNATAKA,1,0,0,0
2,ALLAHABAD BANK,ALLA0210001,GUWAHATI MAIN BRANCH,"H. B. ROAD, MACHOWA, OPP. OF ASTC BUS STAND MA...",2545218,GUWAHATI,KAMRUP,ASSAM,0,0,1,0
3,ALLAHABAD BANK,ALLA0210002,PATNA UNIVERSITY,"PATNA UNIVERSITY.GANESH ROAD,DIST-PATNA",2671644,PATNA,PATNA,BIHAR,0,0,1,0
4,ALLAHABAD BANK,ALLA0210003,PATNA MAIN,"BUDH MARG,DIST-PATNA,PIN-800001",2235237,PATNA,PATNA,BIHAR,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
121035,YES BANK,YESB0ZSBL20,ZILA SAHKARI BANK LUCKNOW ICCMRT EC,"21/467, INDIRA NAGAR, LUCKNOW - 226016",8756400236.0,LUCKNOW,LUCKNOW,UTTAR PRADESH,0,0,0,0
121036,YES BANK,YESB0ZSBL21,ZILA SAHKARI BANK LUCKNOW PARAGDARY,"JOPPLING ROAD, LUCKNOW - 226001",8756400237.0,LUCKNOW,LUCKNOW,UTTAR PRADESH,0,0,0,0
121037,YES BANK,YESB0ZSBL22,ZILA SAHKARI BANK LUCKNOW ARJUNGANJ,"AHMAMAU, ARJUNGANJ, LUCKNOW - 226002",8756400239.0,ARJUNGANJ,LUCKNOW,UTTAR PRADESH,0,0,0,0
121038,YES BANK,YESB0ZSBL23,ZILA SAHKARI BANK LUCKNOW CHANDRNGR,"H NO 551 KA/180 KHA,ST BANK RD, BHILLAWAN,CHAN...",8756400238.0,LUCKNOW,LUCKNOW,UTTAR PRADESH,0,0,0,0


In [36]:
#branch_all.to_csv(f'/Users/uddhav/Desktop/Thesis/bank_branch_csv/branch_all.csv')

In [37]:
with open('/Users/uddhav/Desktop/Thesis/by-bank_cov/banknames.json', 'r') as file_3:
    data_3 = json.load(file_3)

In [38]:
bank_branch_code2 = pd.DataFrame([data_3])
bank_branch_code2 = bank_branch_code2.T
bank_branch_code2 = bank_branch_code2.reset_index()
bank_branch_code2 = bank_branch_code2.rename(columns = { 'index' : 'Code', 0 : 'bank_name'})
branch_name_list2 = bank_branch_code2['bank_name'].to_list()
branch_name_list_cap2 = [x.upper() for x in branch_name_list2]
branch_name_list_cap2 = sorted(branch_name_list_cap2)
branch_name_list_cap2

['510 ARMY BASE W/S CREDIT CO-OPERATIVE PRIMARY BANK',
 'A.P. MAHESH CO-OPERATIVE URBAN BANK',
 'AB BANK',
 'ABHINANDAN URBAN CO-OPERATIVE BANK AMRAVATI',
 'ABHINAV SAHAKARI BANK',
 'ABHYUDAYA CO-OPERATIVE BANK',
 'ABU DHABI COMMERCIAL BANK',
 'ACE CO-OPERATIVE BANK',
 'ADARNIYA P.D. PATILSAHEB SAHAKARI BANK',
 'ADARSH CO-OPERATIVE BANK',
 'ADARSH CO-OPERATIVE URBAN BANK',
 'ADARSH MAHILA MERCANTILE CO-OPERATIVE BANK',
 'ADILABAD DISTRICT CO-OPERATIVE CENTRAL BANK',
 'ADINATH CO-OPERATIVE BANK',
 'ADITYA BIRLA IDEA PAYMENTS BANK',
 'ADV.SHAMRAOJI SHINDE SATYASHODHAK BANK',
 'AGARTALA CO-OPERATIVE URBAN BANK',
 'AGRA DISTRICT CO-OPERATIVE BANK',
 'AGRASEN CO-OPERATIVE URBAN BANK',
 'AGRASEN NAGARI SAHAKARI BANK',
 'AGROHA CO-OPERATIVE URBAN BANK',
 'AHILYADEVI URBAN CO-OPERATIVE BANK LIMITED SOLAPUR',
 'AHMEDABAD DISTRICT CO-OPERATIVE BANK',
 'AHMEDABAD MERCANTILE CO-OPERATIVE BANK',
 'AHMEDNAGAR DISTRICT CENTRAL CO-OPERATIVE BANK',
 "AHMEDNAGAR MERCHANT'S CO-OPERATIVE BANK",
 'AHMEDNAG

In [39]:
branch_name_list_cap2_1 = branch_name_list_cap2[-500:]
branch_name_list_cap2_1

['PARBHANI DISTRICT CENTRAL CO-OPERATIVE BANK',
 'PARSHWANATH CO-OPERATIVE BANK',
 'PARWANOO URBAN CO-OPERATIVE BANK',
 'PASCHIM BANGA GRAMIN BANK',
 'PATAN CO-OPERATIVEEARTIVE BANK',
 'PATAN NAGARIK SAHAKARI BANK',
 'PATAN URBAN CO-OPERATIVE BANK PATAN',
 'PATHANMTHITTA DISTRICT CO-OPERATIVE BANK',
 'PATIALA CENTRAL CO-OPERATIVE BANK',
 'PATLIPUTRA CENTRAL CO-OPERATIVE BANK',
 'PAVANA SAHAKARI BANK',
 'PAYANGADI URBAN CO-OPERATIVE BANK',
 'PAYTM PAYMENTS BANK',
 'PAYYOLI CO-OPERATIVE URBAN BANK',
 'PEOPLES URBAN CO-OPERATIVE BANK',
 "PEOPLES' CO-OPERATIVE BANK",
 'PIMPALGAON MERCHANTS CO-OPERATIVE BANK',
 'PIMPRI CHINCHWAD SAHAKARI BANK MARYADIT,PIMPRI',
 'PITHORAGARH ZILA SAHKARI BANK',
 'POCHAMPALLY CO-OPERATIVE URBAN BANK',
 'PONDICHERRY STATE CO-OPERATIVE BANK',
 'POORNAWADI NAGRIK SAHAKARI BANK',
 'PRAGATHI CO-OPERATIVE BANK, BANGALORE',
 'PRAGATHI KRISHNA GRAMIN BANK',
 'PRAGATI CO-OPERATIVE BANK,THARA',
 'PRAGATI MAHILA NAGRIK SAHAKARI BANK BHILAI',
 'PRAGATI SAHAKARI BANK',
 '

In [40]:
len(branch_name_list_cap2)

1431

In [41]:
bank_branch_code2

,Code,bank_name
0,AACX,Akhand Anand Co.op Bank
1,ABBL,AB Bank
2,ABCX,Aurangabad District Central Co-operative Bank
3,ABDX,Dr. Ambedkar Nagrik Sahakari Bank Mydt Gwalior
4,ABEX,Andhra Bank Employees Co-operative Bank
...,...,...
1426,ZSHX,Zila Sahkari Bank Haridwar
1427,ZSJX,Zila Sahkari Bank Jhansi
1428,ZSKX,Zila Sahkari Bank Kanpur
1429,ZSLX,Zila Sahkari Bank Lakhimpur Kheri


In [42]:
inter_11 = []
for i in All_Banks_3:
    if i in branch_name_list_cap2:
        inter_11.append(i)

len(inter_11)

41

In [43]:
inter_21 = []
for i in All_Banks_2:
    if i in branch_name_list_cap2:
        inter_21.append(i)
len(inter_21)

59

In [44]:
Foreign_Banks_3 = ["AB BANK", "ABU DHABI COMMERCIAL BANK" ,"AMERICAN EXPRESS BANKING CORP", "AUSTRALIA AND NEW ZEALAND BANKING GROUP" ,"BANK OF AMERICA" ,"BANK OF BAHREIN AND KUWAIT" ,"BANK OF CEYLON", "BANK OF NOVA SCOTIA", "BARCLAYS BANK" ,"BNP PARIBAS BANK" ,"CITI BANK" ,"COMMONWEALTH BANK OF AUSTRALIA", "COOPERATIEVE RABOBANK U.A." ,"CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK", "CREDIT SUISSE AG", "CTBC BANK CO., LTD" ,"DBS BANK INDIA LTD", "DEUTSCHE BANK" ,"FIRSTRAND BANK" ,"HONGKONG & SHANGHAI BANKING CORPORATION" ,"HSBC BANK OMAN S.A.O.G." ,"INDUSTRIAL AND COMMERCIAL BANK OF CHINA", "JP MORGAN CHASE BANK NA", "JSC VTB BANK", "KBC BANK NV" ,"KRUNGTHAI BANK" ,"MASHREQ BANK", "MIZUHO BANK", "MUFG BANK LTD", "NATIONAL AUSTRALIA BANK", "NatWest Markets Plc", "SBER BANK" ,"SBM BANK (INDIA) LTD" ,"SHINHAN BANK" ,"SOCIETE GENERALE" ,"SONALI BANK", "STANDARD CHARTERED BANK" ,"UBS AG" ,"UNITED OVERSEAS BANK" ,"WOORI BANK", "SUMITOMO MITSUI BANKING CORPORATION", "WESTPAC BANKING CORPORATION", "PT BANK MAYBANK INDONESIA TBK", "DOHA BANK QSC", "INDUSTRIAL BANK OF KOREA", "FIRST ABU DHABI BANK PJSC", "KEB HANA BANK", "Qatar National Bank SAQ", "EMIRATES NBD BANK", "BANK OF CHINA LTD", "KOOKMIN BANK"]
Private_Banks_3 = ["AXIS BANK", "CITY UNION BANK", "CATHOLIC SYRIAN BANK", "DCB BANK", "FEDERAL BANK", "HDFC BANK", "ICICI BANK", "IDBI", "INDUSIND BANK", "ING VYSYA BANK", "JAMMU AND KASHMIR BANK", "KARNATAKA BANK" ,"KARUR VYSYA BANK" ,"KOTAK MAHINDRA BANK", "LAXMI VILAS BANK" ,"NAINITAL BANK LTD", "RBL BANK", "SOUTH INDIAN BANK", "TAMILNADU MERCANTILE BANK", "DHANLAXMI BANK" ,"YES BANK", "IDFC FIRST BANK", "BANDHAN BANK"]
Public_Banks_3 = ["ALLAHABAD BANK", "ANDHRA BANK", "BANK OF BARODA", "BANK OF INDIA", "BANK OF MAHARASHTRA", "CANARA BANK", "CENTRAL BANK OF INDIA", "CORPORATION BANK", "DENA BANK", "IDBI", "INDIAN BANK", "INDIAN OVERSEAS BANK", "ORIENTAL BANK OF COMMERCE", "PUNJAB & SIND BANK", "PUNJAB NATIONAL BANK", "STATE BANK OF BIKANER AND JAIPUR", "STATE BANK OF HYDERABAD", "STATE BANK OF INDIA", "STATE BANK OF MYSORE", "STATE BANK OF PATIALA", "STATE BANK OF TRAVANCORE", "SYNDICATE BANK", "UCO BANK", "UNION BANK OF INDIA", "UNITED BANK OF INDIA", "VIJAYA BANK", "BHARATIYA MAHILA BANK"]
Small_Finance_3= ["CAPITAL SMALL FINANCE BANK", "EQUITAS SMALL FINANCE BANK", "ESAF SMALL FINANCE BANK", "FINCARE SMALL FINANCE BANK", "SURYODAY SMALL FINANCE BANK", "UJJIVAN SMALL FINANCE BANK", "AU SMALL FINANCE BANK", "NORTH EAST SMALL FINANCE BANK", "UTKARSH SMALL FINANCE BANK", "JANA SMALL FINANCE BANK"]
All_Banks_4 = Foreign_Banks_3 + Private_Banks_3 + Public_Banks_3 + Small_Finance_3
All_Banks_4 = [x.upper() for x in All_Banks_4]
All_Banks_4

['AB BANK',
 'ABU DHABI COMMERCIAL BANK',
 'AMERICAN EXPRESS BANKING CORP',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF CEYLON',
 'BANK OF NOVA SCOTIA',
 'BARCLAYS BANK',
 'BNP PARIBAS BANK',
 'CITI BANK',
 'COMMONWEALTH BANK OF AUSTRALIA',
 'COOPERATIEVE RABOBANK U.A.',
 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK',
 'CREDIT SUISSE AG',
 'CTBC BANK CO., LTD',
 'DBS BANK INDIA LTD',
 'DEUTSCHE BANK',
 'FIRSTRAND BANK',
 'HONGKONG & SHANGHAI BANKING CORPORATION',
 'HSBC BANK OMAN S.A.O.G.',
 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA',
 'JP MORGAN CHASE BANK NA',
 'JSC VTB BANK',
 'KBC BANK NV',
 'KRUNGTHAI BANK',
 'MASHREQ BANK',
 'MIZUHO BANK',
 'MUFG BANK LTD',
 'NATIONAL AUSTRALIA BANK',
 'NATWEST MARKETS PLC',
 'SBER BANK',
 'SBM BANK (INDIA) LTD',
 'SHINHAN BANK',
 'SOCIETE GENERALE',
 'SONALI BANK',
 'STANDARD CHARTERED BANK',
 'UBS AG',
 'UNITED OVERSEAS BANK',
 'WOORI BANK',
 'SUMITOMO MITSUI BANKING CORPORATION',
 '

In [45]:
inter_31 = []
for i in All_Banks_4:
    if i in branch_name_list_cap2:
        inter_31.append(i)
len(inter_31)

96

In [46]:
inter_41 = []
for i in branch_name_list_cap2:
    if i in All_Banks_4:
        inter_41.append(i)
len(inter_41)

98

In [47]:
bank_branch_code2['bank_name'] = bank_branch_code2['bank_name'].str.upper()
bank_branch_code2

,Code,bank_name
0,AACX,AKHAND ANAND CO.OP BANK
1,ABBL,AB BANK
2,ABCX,AURANGABAD DISTRICT CENTRAL CO-OPERATIVE BANK
3,ABDX,DR. AMBEDKAR NAGRIK SAHAKARI BANK MYDT GWALIOR
4,ABEX,ANDHRA BANK EMPLOYEES CO-OPERATIVE BANK
...,...,...
1426,ZSHX,ZILA SAHKARI BANK HARIDWAR
1427,ZSJX,ZILA SAHKARI BANK JHANSI
1428,ZSKX,ZILA SAHKARI BANK KANPUR
1429,ZSLX,ZILA SAHKARI BANK LAKHIMPUR KHERI


In [48]:
bank_branch_code_filter2 = bank_branch_code2[bank_branch_code2['bank_name'].isin(inter_31)]
bank_branch_code_filter2

,Code,bank_name
1,ABBL,AB BANK
18,ADCB,ABU DHABI COMMERCIAL BANK
44,ALLA,ALLAHABAD BANK
58,ANDB,ANDHRA BANK
63,ANZB,AUSTRALIA AND NEW ZEALAND BANKING GROUP
...,...,...
1338,UTKS,UTKARSH SMALL FINANCE BANK
1363,VIJB,VIJAYA BANK
1387,VYSA,ING VYSYA BANK
1397,WPAC,WESTPAC BANKING CORPORATION


In [49]:
bank_branch_code_filter2 = bank_branch_code_filter2[~bank_branch_code_filter2['Code'].isin(['ABBL', 'BMBL', 'CTBA', 'ESAF', 'FIRX', 'NATA', 'SBBJ', 'SBHY', 'SBLD', 'SBMY', 'SBTR', 'STBP', 'USFB', 'VYSA'])]

In [50]:
banks_to_read2 = bank_branch_code_filter2['Code']
banks_to_read2

18      ADCB
44      ALLA
58      ANDB
63      ANZB
85      AUBL
        ... 
1337    UTIB
1338    UTKS
1363    VIJB
1397    WPAC
1406    YESB
Name: Code, Length: 84, dtype: object

In [138]:
sorted(bank_branch_code_filter2['bank_name'].to_list())

['ABU DHABI COMMERCIAL BANK',
 'ALLAHABAD BANK',
 'ANDHRA BANK',
 'AU SMALL FINANCE BANK',
 'AUSTRALIA AND NEW ZEALAND BANKING GROUP',
 'AXIS BANK',
 'BANDHAN BANK',
 'BANK OF AMERICA',
 'BANK OF BAHREIN AND KUWAIT',
 'BANK OF BARODA',
 'BANK OF CEYLON',
 'BANK OF INDIA',
 'BANK OF MAHARASHTRA',
 'BANK OF NOVA SCOTIA',
 'BARCLAYS BANK',
 'BNP PARIBAS BANK',
 'CANARA BANK',
 'CAPITAL SMALL FINANCE BANK',
 'CATHOLIC SYRIAN BANK',
 'CENTRAL BANK OF INDIA',
 'CITI BANK',
 'CITY UNION BANK',
 'CORPORATION BANK',
 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK',
 'CREDIT SUISSE AG',
 'DCB BANK',
 'DENA BANK',
 'DEUTSCHE BANK',
 'DHANLAXMI BANK',
 'DOHA BANK QSC',
 'EMIRATES NBD BANK',
 'EQUITAS SMALL FINANCE BANK',
 'ESAF SMALL FINANCE BANK',
 'FEDERAL BANK',
 'FINCARE SMALL FINANCE BANK',
 'FIRSTRAND BANK',
 'HDFC BANK',
 'HONGKONG & SHANGHAI BANKING CORPORATION',
 'ICICI BANK',
 'IDBI',
 'IDFC FIRST BANK',
 'INDIAN BANK',
 'INDIAN OVERSEAS BANK',
 'INDUSIND BANK',
 'INDUSTRIAL AND COMMERCI

In [52]:
data_dict2 = {}
for i in banks_to_read2:
    with open(f'/Users/uddhav/Desktop/Thesis/by-bank_cov/by-bank 2/{i}.json', 'r') as file:
        data_dict2[i] = json.load(file)

In [53]:
len(data_dict2)

84

In [54]:
for i in banks_to_read2:
    globals()[f'bank_branch2_{i}'] = pd.json_normalize(data_dict2[i].values())

In [55]:
bank_branch2_HDFC

,IFSC,BRANCH,CENTRE,DISTRICT,STATE,ADDRESS,CONTACT,MICR,BANK,UPI,RTGS,CITY,NEFT,IMPS
0,HDFC0000001,TULSIANI CHMBRS - NARIMAN PT,MUMBAI,GREATER MUMBAI,MAHARASHTRA,101-104 TULSIANI CHAMBERSFREE PRESS JOURNAL MA...,9890603333,400240003,HDFC BANK,True,True,MUMBAI,True,True
1,HDFC0000002,MUMBAI - KHAR (WEST),MUMBAI,GREATER MUMBAI,MAHARASHTRA,"SWAGATAM,OPP. KHAR POLICE STATION,S.V.ROAD, KH...",9890603333,400240004,HDFC BANK,True,True,MUMBAI,True,True
2,HDFC0000003,K G MARG,DELHI,NEW DELHI,DELHI,"209 - 214, KAILASH BUILDING, 26, KASTURBA GAND...",61606161,110240001,HDFC BANK,True,True,DELHI,True,True
3,HDFC0000004,CHENNAI - ITC CENTRE - ANNA SALAI,CHENNAI,CHENNAI,TAMIL NADU,"759, ITC CENTREANNA SALAI, OPP T.V.S.CHENNAITA...",9840673333,600240002,HDFC BANK,True,True,CHENNAI,True,True
4,HDFC0000005,BHULABHAI DESAI ROAD,MUMBAI,GREATER MUMBAI,MAHARASHTRA,HDFC BANK LTD 36 VALLABH APTS 87 BHULABHAI DES...,61606161,400240005,HDFC BANK,True,True,MUMBAI,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6740,HDFC0CYACBP,Yashwant Co-operative Bank IMPS,KARAD,SATARA,MAHARASHTRA,"69,GURUWAR PETH,CHAWDI CHOWK,KARAD 415110",9422745818,415814352,Yashwant Co-operative Bank,True,True,KARAD,True,True
6741,HDFC0CYDCBL,YOUTH DEVELOPMENT COOP BANK LTD.KOL,KOLHAPUR,KOLHAPUR,MAHARASHTRA,"""POORNIMA"" F-2, FIRST FLOOR, 1429/G, NEAR SAMB...",2550120,416393015,HDFC BANK,True,True,KOLHAPUR,True,True
6742,HDFC0CYMCBK,Yeola mer co o pbank ltd,YEOLA,NASIK,MAHARASHTRA,Yashodhan Balaji Galli Yeola Dist Nasik,02559265047,423853352,HDFC BANK,True,True,YEOLA,True,True
6743,HDFC0CYMSB1,Yavatmal Mahila Sahakari Bank IMPS,YAVATMAL,YAVATMAL,MAHARASHTRA,"Annapurna, Mainde Chowk, Sankat Mochan Road Ya...",07232250661,445814002,Yavatmal Mahila Sahakari Bank,True,True,YAVATMAL,True,True


In [56]:
branch_all2 = pd.concat([globals()[f'bank_branch2_{i}'] for i in banks_to_read2], ignore_index=True)
branch_all2['foreign_dummy'] = branch_all2['BANK'].isin(Foreign_Banks_3).astype(int)
branch_all2['private_dummy'] = branch_all2['BANK'].isin(Private_Banks_3).astype(int)
branch_all2['public_dummy'] = branch_all2['BANK'].isin(Public_Banks_3).astype(int)
branch_all2['small_finance_dummy'] = branch_all2['BANK'].isin(Small_Finance_3).astype(int)

In [57]:
branch_all2

,BANK,IFSC,BRANCH,CENTRE,DISTRICT,STATE,ADDRESS,CONTACT,IMPS,CITY,UPI,RTGS,NEFT,MICR,foreign_dummy,private_dummy,public_dummy,small_finance_dummy
0,Abu Dhabi Commercial Bank,ADCB0000001,Abu Dhabi Commercial Bank IMPS,MUMBAI,GREATER BOMBAY,MAHARASHTRA,"75 Rehmat Manzil, Veer Nariman Road, Churchgat...",39534100,True,MUMBAI,False,True,True,None,0,0,0,0
1,ABU DHABI COMMERCIAL BANK,ADCB0000002,BANGALORE,NA,BANGALORE,KARNATAKA,"CITI CENTRE, 28, CHURCH STREET, OFF M. G. ROAD...",25582000,True,BANGALORE URBAN,False,False,True,None,1,0,0,0
2,ALLAHABAD BANK,ALLA0210001,Guwahati Main Branch,GUWAHATI,KAMRUP METROPOLITAN,ASSAM,"H. B. Road,Machkhowa, Opp. Of ASTC Bus Stand M...",2545218,True,GUWAHATI,True,True,True,None,0,0,1,0
3,ALLAHABAD BANK,ALLA0210002,PATNA UNIVERSITY,PATNA,PATNA,BIHAR,"PATNA UNIVERSITY.GANESH ROAD,DIST-PATNA",2671644,True,PATNA,True,True,True,None,0,0,1,0
4,ALLAHABAD BANK,ALLA0210003,PATNA MAIN,PATNA,PATNA,BIHAR,"BUDH MARG,DIST-PATNA,PIN-800001",2235237,True,PATNA,True,True,True,None,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138639,YES BANK,YESB0ZSBL20,ZILA SAHKARI BANK LUCKNOW ICCMRT EC,LUCKNOW,LUCKNOW,UTTAR PRADESH,"21/467, INDIRA NAGAR, LUCKNOW - 226016",8756400236,True,LUCKNOW,True,True,True,226185254,0,1,0,0
138640,YES BANK,YESB0ZSBL21,ZILA SAHKARI BANK LUCKNOW PARAGDARY,LUCKNOW,LUCKNOW,UTTAR PRADESH,"JOPPLING ROAD, LUCKNOW - 226001",8756400237,True,LUCKNOW,True,True,True,226185258,0,1,0,0
138641,YES BANK,YESB0ZSBL22,ZILA SAHKARI BANK LUCKNOW ARJUNGANJ,ARJUNGANJ,LUCKNOW,UTTAR PRADESH,"AHMAMAU, ARJUNGANJ, LUCKNOW - 226002",8756400239,True,ARJUNGANJ,True,True,True,226185252,0,1,0,0
138642,YES BANK,YESB0ZSBL23,ZILA SAHKARI BANK LUCKNOW CHANDRNGR,LUCKNOW,LUCKNOW,UTTAR PRADESH,"H NO 551 KA/180 KHA,ST BANK RD, BHILLAWAN,CHAN...",8756400238,True,LUCKNOW,True,True,True,226185257,0,1,0,0


In [58]:
#branch_all2.to_csv(f'/Users/uddhav/Desktop/Thesis/bank_branch_csv/branch_all_cov.csv')

In [59]:
Private_Banks_eq = {'AXIS BANK LIMITED' : 'AXISBANK.BO', 'CITY UNION BANK LIMITED' : 'CUB.BO', 'CSB BANK LIMITED' : 'CSBBANK.BO', 'DCB BANK LIMITED' : 'DCBBANK.BO', 'FEDERAL BANK LTD' : 'FEDERALBNK.BO', 'HDFC BANK LTD.' : 'HDFCBANK.BO', 'ICICI BANK LIMITED':'ICICIBANK.BO', 'INDUSIND BANK LTD':'INDUSINDBK.BO', 'JAMMU & KASHMIR BANK LTD':'J&KBANK.BO', 'KARNATAKA BANK LTD':'KTKBANK.BO', 'KARUR VYSYA BANK LTD':'KARURVYSYA.BO', 'KOTAK MAHINDRA BANK LTD.':'KOTAKBANK.BO', 'RBL BANK LIMITED':'RBLBANK.BO', 'SOUTH INDIAN BANK LTD':'SOUTHBANK.BO', 'THE DHANALAKSHMI BANK LTD':'DHANBANK.BO', 'YES BANK LTD.':'YESBANK.BO', 'IDFC FIRST BANK LIMITED':'IDFCFIRSTB.BO', 'BANDHAN BANK LIMITED':'BANDHANBNK.BO'}
Public_Banks_eq = {'BANK OF BARODA':'BANKBARODA.BO', 'BANK OF INDIA':'BANKINDIA.BO', 'BANK OF MAHARASHTRA':'MAHABANK.BO', 'CANARA BANK':'CANBK.BO', 'CENTRAL BANK OF INDIA':'CENTRALBK.BO', 'IDBI BANK LIMITED':'IDBI.BO', 'INDIAN BANK':'INDIANB.BO', 'INDIAN OVERSEAS BANK':'IOB.BO', 'PUNJAB NATIONAL BANK':'PNB.BO', 'STATE BANK OF INDIA':'SBIN.BO', 'UCO BANK':'UCOBANK.BO', 'UNION BANK OF INDIA':'UNIONBANK.BO'}
Small_Finance_eq = {'UJJIVAN SMALL FINANCE BANK LIMITED':'UJJIVANSFB.BO'}
Foreign_Banks_eq= {'AMERICAN EXPRESS BANKING CORP.':'AXP', 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED':'AN3PH.AX', 'BANK OF AMERICA , NATIONAL ASSOCIATION':'BAC', 'BANK OF NOVA SCOTIA':'BNS', 'BNP PARIBAS':'BNP.PA', 'CITIBANK N.A':'C', 'COMMONWEALTH BANK OF AUSTRALIA':'CBA.AX', 'DEUTSCHE BANK AG':'DB', 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA':'1398.HK', 'JPMORGAN CHASE BANK NATIONAL ASSOCIATION':'JPM-PD', 'KRUNG THAI BANK PUBLIC COMPANY LIMITED':'KTB.BK', 'NATIONAL AUSTRALIA BANK':'NAB.AX', 'SOCIETE GENERALE':'GLE.PA', 'STANDARD CHARTERED BANK':'STAN.L', 'UBS AG':'UBS', 'UNITED OVERSEAS BANK LTD':'UOVEF', 'WESTPAC BANKING CORPORATION':'WBC.AX', 'PT BANK MAYBANK INDONESIA TBK':'BNII.JK', 'DOHA BANK Q.P.S.C':'DHBK.QA', 'INDUSTRIAL BANK OF KOREA':'024110.KS', 'Qatar National Bank (Q.P.S.C.)':'QNBK.QA', 'EMIRATES NBD BANK (P.J.S.C.)':'EMIRATESNBD.AE', 'BANK OF CHINA LIMITED':'3988.HK'}
Foreign_Banks_country = {'AMERICAN EXPRESS BANKING CORP.':'US', 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED':'AU', 'BANK OF AMERICA , NATIONAL ASSOCIATION':'US', 'BANK OF NOVA SCOTIA':'US', 'BNP PARIBAS':'EU', 'CITIBANK N.A':'US', 'COMMONWEALTH BANK OF AUSTRALIA':'AU', 'DEUTSCHE BANK AG':'US', 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA':'HK', 'JPMORGAN CHASE BANK NATIONAL ASSOCIATION':'US', 'KRUNG THAI BANK PUBLIC COMPANY LIMITED':'TH', 'NATIONAL AUSTRALIA BANK':'AU', 'SOCIETE GENERALE':'EU', 'STANDARD CHARTERED BANK':'GB', 'UBS AG':'US', 'UNITED OVERSEAS BANK LTD':'US', 'WESTPAC BANKING CORPORATION':'AU', 'PT BANK MAYBANK INDONESIA TBK':'IN', 'DOHA BANK Q.P.S.C':'QA', 'INDUSTRIAL BANK OF KOREA':'SK', 'Qatar National Bank (Q.P.S.C.)':'QA', 'EMIRATES NBD BANK (P.J.S.C.)':'UAE', 'BANK OF CHINA LIMITED':'HK'}

In [60]:
Private_Banks_no_eq = ['ING VYSYA BANK LTD', 'LAKSHMI VILAS BANK LTD', 'NAINITAL BANK LTD', 'TAMILNAD MERCANTILE BANK LTD']
Public_Banks_no_eq = ['ALLAHABAD BANK', 'ANDHRA BANK', 'CORPORATION BANK', 'DENA BANK', 'ORIENTAL BANK OF COMMERCE', 'PUNJAB AND SIND BANK', 'STATE BANK OF BIKANER AND JAIPUR', 'STATE BANK OF MYSORE', 'STATE BANK OF PATIALA', 'STATE BANK OF TRAVANCORE', 'SYNDICATE BANK', 'UNITED BANK OF INDIA', 'VIJAYA BANK', 'BHARATIYA MAHILA BANK LTD.']
Small_Finance_no_eq = ['CAPITAL SMALL FINANCE BANK LIMITED', 'FINCARE SMALL FINANCE BANK LIMITED', 'NORTH EAST SMALL FINANCE BANK LIMITED', 'JANA SMALL FINANCE BANK LIMITED', 'EQUITAS SMALL FINANCE BANK LIMITED', 'ESAF SMALL FINANCE BANK LIMITED', 'SURYODAY SMALL FINANCE BANK LIMITED', 'UTKARSH SMALL FINANCE BANK LIMITED']
Foreign_Banks_no_eq = ['AB BANK LIMITED', 'ABU DHABI COMMERCIAL BANK PJSC', 'BANK OF BAHRAIN & KUWAIT B.S.C.', 'BANK OF CEYLON', 'BARCLAYS BANK PLC', 'COOPERATIEVE RABOBANK U.A.', 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK', 'CREDIT SUISSE AG', 'CTBC BANK CO., LTD.', 'DBS BANK INDIA LTD.', 'FIRSTRAND BANK LTD', 'HONGKONG AND SHANGHAI BANKING CORPN.LTD.', 'HSBC BANK OMAN S.A.O.G.', 'JSC VTB BANK', 'KBC BANK NV', 'MASHREQ BANK PSC', 'MIZUHO BANK LTD', 'MUFG Bank Ltd', 'NatWest Markets Plc', 'SBERBANK', 'SBM BANK (INDIA) LTD.','SHINHAN BANK', 'SONALI BANK', 'WOORI BANK', 'SUMITOMO MITSUI BANKING CORPORATION', 'FIRST ABU DHABI BANK PJSC', 'KEB HANA BANK', 'KOOKMIN BANK']

In [61]:
for j, i in Private_Banks_eq.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2019-12-31')

for i in Private_Banks_eq.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    temp_df['Country'] = "IND"
    globals()[f'{i}_EQ'] = temp_df

# for i in Private_Banks_eq.keys():
#     globals()[f'{i}_EQ'].to_csv(f'/Users/uddhav/Desktop/Thesis/bank_equity_csv/{i}_equity.csv')

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2384286038.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2384286038.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2384286038.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2384286038.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/238

In [62]:
for j, i in Public_Banks_eq.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2019-12-31')

for i in Public_Banks_eq.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    temp_df['Country'] = "IND"
    globals()[f'{i}_EQ'] = temp_df

# for i in Public_Banks_eq.keys():
#     globals()[f'{i}_EQ'].to_csv(f'/Users/uddhav/Desktop/Thesis/bank_equity_csv/{i}_equity.csv')

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2494448997.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2494448997.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2494448997.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2494448997.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/249

In [63]:
for j, i in Small_Finance_eq.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2019-12-31')

for i in Small_Finance_eq.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    temp_df['Country'] = "IND"
    globals()[f'{i}_EQ'] = temp_df

# for i in Small_Finance_eq.keys():
#     globals()[f'{i}_EQ'].to_csv(f'/Users/uddhav/Desktop/Thesis/bank_equity_csv/{i}_equity.csv')

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/1241886761.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')


In [64]:
for j, i in Foreign_Banks_eq.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2019-12-31')

for i in Foreign_Banks_eq.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    globals()[f'{i}_EQ'] = temp_df

for i,j in Foreign_Banks_country.items():
    globals()[f'{i}_EQ']['Country'] = j

# for i in Foreign_Banks_eq.keys():
#     globals()[f'{i}_EQ'].to_csv(f'/Users/uddhav/Desktop/Thesis/bank_equity_csv/{i}_equity.csv')

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2156737910.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2156737910.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2156737910.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2156737910.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/215

In [65]:
s_p = yf.Ticker('BSE-500.BO')
s_p = s_p.history(start='2012-06-30', end='2019-12-31')
s_p['Bank'] = 'S&P500'
s_p['Country'] = 'IND'
s_p = s_p.reset_index()
s_p['Date'] = s_p['Date'].dt.to_period(freq='D')
s_p

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/94887499.py:6: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  s_p['Date'] = s_p['Date'].dt.to_period(freq='D')


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Bank,Country
0,2012-07-02,6695.990234,6724.910156,6677.089844,6700.359863,0,0.0,0.0,S&P500,IND
1,2012-07-03,6726.950195,6751.740234,6701.700195,6726.859863,0,0.0,0.0,S&P500,IND
2,2012-07-04,6752.970215,6758.959961,6716.209961,6751.189941,0,0.0,0.0,S&P500,IND
3,2012-07-05,6748.779785,6797.049805,6742.140137,6787.470215,0,0.0,0.0,S&P500,IND
4,2012-07-06,6786.490234,6786.490234,6743.430176,6766.750000,0,0.0,0.0,S&P500,IND
...,...,...,...,...,...,...,...,...,...,...
1814,2019-12-23,15691.429688,15744.049805,15649.730469,15705.980469,104300,0.0,0.0,S&P500,IND
1815,2019-12-24,15724.950195,15735.299805,15647.969727,15659.030273,80400,0.0,0.0,S&P500,IND
1816,2019-12-26,15688.139648,15688.139648,15571.839844,15581.540039,92800,0.0,0.0,S&P500,IND
1817,2019-12-27,15623.809570,15737.790039,15619.339844,15726.459961,93100,0.0,0.0,S&P500,IND


In [66]:
Private_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Private_Banks_eq.keys()], ignore_index=True)
Private_eq_merge['foreign_dummy'] = 0
Private_eq_merge['private_dummy'] = 1
Private_eq_merge['public_dummy'] = 0
Private_eq_merge['small_finance_dummy'] = 0
Public_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Public_Banks_eq.keys()], ignore_index=True)
Public_eq_merge['foreign_dummy'] = 0
Public_eq_merge['private_dummy'] = 0
Public_eq_merge['public_dummy'] = 1
Public_eq_merge['small_finance_dummy'] = 0
Small_Finance_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Small_Finance_eq.keys()], ignore_index=True)
Small_Finance_eq_merge['foreign_dummy'] = 0
Small_Finance_eq_merge['private_dummy'] = 0
Small_Finance_eq_merge['public_dummy'] = 0
Small_Finance_eq_merge['small_finance_dummy'] = 1
Foreign_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Foreign_Banks_eq.keys()], ignore_index=True)
Foreign_eq_merge['foreign_dummy'] = 1
Foreign_eq_merge['private_dummy'] = 0
Foreign_eq_merge['public_dummy'] = 0
Foreign_eq_merge['small_finance_dummy'] = 0
Equity_all = pd.concat([Private_eq_merge, Public_eq_merge, Small_Finance_eq_merge, Foreign_eq_merge, s_p])

In [67]:
Equity_all['Country'].unique()

array(['IND', 'US', 'AU', 'EU', 'HK', 'TH', 'GB', 'IN', 'QA', 'SK', 'UAE'],
      dtype=object)

In [68]:
Currency_ticks = {'US':'USDINR=X', 'AU':'AUDINR=X', 'EU':'EURINR=X', 'HK':'HKDINR=X', 'TH':'THBINR=X', 'GB':'GBPINR=X', 'IN':'IDRINR=X', 'QA':'QARUSD=X', 'SK':'KRWINR=X', 'UAE':'AEDINR=X'}
for j, i in Currency_ticks.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2019-12-31')

In [69]:
for i in Currency_ticks.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Country'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    globals()[f'{i}_EQ'] = temp_df

# for i in Currency_ticks.keys():
#     globals()[f'{i}_EQ'].to_csv(f'/Users/uddhav/Desktop/Thesis/bank_equity_csv/{i}_forex.csv')

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/754670561.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/754670561.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/754670561.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/754670561.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/7546705

In [70]:
QA_EQ = pd.merge(globals()["QA_EQ"], globals()["US_EQ"], on='Date')
QA_EQ['Open'] = QA_EQ['Open_x']*QA_EQ['Open_y']
QA_EQ['High'] = QA_EQ['High_x']*QA_EQ['High_y']
QA_EQ['Low'] = QA_EQ['Low_x']*QA_EQ['Low_y']
QA_EQ['Close'] = QA_EQ['Close_x']*QA_EQ['Close_y']
QA_EQ

,Date,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,Country_x,Open_y,...,Low_y,Close_y,Volume_y,Dividends_y,Stock Splits_y,Country_y,Open,High,Low,Close
0,2012-07-02,0.284884,0.284884,0.274688,0.274801,0,0.0,0.0,QA,55.570000,...,55.349998,55.502998,0,0.0,0.0,US,15.831006,15.925018,15.203954,15.252267
1,2012-07-03,0.274846,0.274975,0.274657,0.274876,0,0.0,0.0,QA,55.428001,...,54.250000,55.428001,0,0.0,0.0,US,15.234169,15.241292,14.900162,15.235844
2,2012-07-04,0.274876,0.274937,0.274816,0.274876,0,0.0,0.0,QA,54.303001,...,54.169998,54.358002,0,0.0,0.0,US,14.926608,15.080281,14.886776,14.941726
3,2012-07-05,0.274891,0.274959,0.274657,0.274876,0,0.0,0.0,QA,54.563000,...,54.423000,54.478001,0,0.0,0.0,US,14.998900,15.183261,14.947678,14.974711
4,2012-07-06,0.274891,0.274944,0.274657,0.274876,0,0.0,0.0,QA,54.938000,...,54.938000,54.938000,0,0.0,0.0,US,15.101984,15.297353,15.089126,15.101154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,2019-12-24,0.274725,0.274725,0.274725,0.274725,0,0.0,0.0,QA,71.209999,...,71.142303,71.210701,0,0.0,0.0,US,19.563185,19.618132,19.544588,19.563378
1947,2019-12-25,0.274725,0.274763,0.274725,0.274725,0,0.0,0.0,QA,71.459999,...,71.187500,71.459999,0,0.0,0.0,US,19.631867,19.634565,19.557004,19.631867
1948,2019-12-26,0.274763,0.274763,0.274725,0.274763,0,0.0,0.0,QA,71.335999,...,71.200500,71.540001,0,0.0,0.0,US,19.600494,19.656547,19.560576,19.656547
1949,2019-12-27,0.274725,0.274725,0.274725,0.274725,0,0.0,0.0,QA,71.459999,...,71.190002,71.300301,0,0.0,0.0,US,19.631867,19.659888,19.557692,19.587994


In [71]:
QA_EQ = QA_EQ[['Date','Open', 'High', 'Low', 'Close']]
QA_EQ['Country'] = "QA"
QA_EQ

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/3012904077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QA_EQ['Country'] = "QA"


,Date,Open,High,Low,Close,Country
0,2012-07-02,15.831006,15.925018,15.203954,15.252267,QA
1,2012-07-03,15.234169,15.241292,14.900162,15.235844,QA
2,2012-07-04,14.926608,15.080281,14.886776,14.941726,QA
3,2012-07-05,14.998900,15.183261,14.947678,14.974711,QA
4,2012-07-06,15.101984,15.297353,15.089126,15.101154,QA
...,...,...,...,...,...,...
1946,2019-12-24,19.563185,19.618132,19.544588,19.563378,QA
1947,2019-12-25,19.631867,19.634565,19.557004,19.631867,QA
1948,2019-12-26,19.600494,19.656547,19.560576,19.656547,QA
1949,2019-12-27,19.631867,19.659888,19.557692,19.587994,QA


In [72]:
INR_EQ = QA_EQ['Date']
INR_EQ = INR_EQ.to_frame()
INR_EQ['Open'] = 1
INR_EQ['High'] = 1
INR_EQ['Low'] = 1
INR_EQ['Close'] = 1
INR_EQ['Country'] = "IND"
INR_EQ

,Date,Open,High,Low,Close,Country
0,2012-07-02,1,1,1,1,IND
1,2012-07-03,1,1,1,1,IND
2,2012-07-04,1,1,1,1,IND
3,2012-07-05,1,1,1,1,IND
4,2012-07-06,1,1,1,1,IND
...,...,...,...,...,...,...
1946,2019-12-24,1,1,1,1,IND
1947,2019-12-25,1,1,1,1,IND
1948,2019-12-26,1,1,1,1,IND
1949,2019-12-27,1,1,1,1,IND


In [73]:
Equity_all.set_index(['Date', 'Country'])
INR_EQ.set_index(['Date', 'Country'])
QA_EQ.set_index(['Date', 'Country'])

,,Open,High,Low,Close
Date,Country,,,,
2012-07-02,QA,15.831006,15.925018,15.203954,15.252267
2012-07-03,QA,15.234169,15.241292,14.900162,15.235844
2012-07-04,QA,14.926608,15.080281,14.886776,14.941726
2012-07-05,QA,14.998900,15.183261,14.947678,14.974711
2012-07-06,QA,15.101984,15.297353,15.089126,15.101154
...,...,...,...,...,...
2019-12-24,QA,19.563185,19.618132,19.544588,19.563378
2019-12-25,QA,19.631867,19.634565,19.557004,19.631867
2019-12-26,QA,19.600494,19.656547,19.560576,19.656547


In [74]:
Equity_all['Country'].unique()

array(['IND', 'US', 'AU', 'EU', 'HK', 'TH', 'GB', 'IN', 'QA', 'SK', 'UAE'],
      dtype=object)

In [75]:
df_1 = pd.merge(Equity_all[Equity_all['Country'] == "IND"], INR_EQ, on= ['Date', 'Country'], how='left')
df_2 = pd.merge(Equity_all[Equity_all['Country'] == "US"], globals()["US_EQ"], on= ['Date', 'Country'], how='left')
df_3 = pd.merge(Equity_all[Equity_all['Country'] == "AU"], globals()["AU_EQ"], on= ['Date', 'Country'], how='left')
df_4 = pd.merge(Equity_all[Equity_all['Country'] == "EU"], globals()["EU_EQ"], on= ['Date', 'Country'], how='left')
df_5 = pd.merge(Equity_all[Equity_all['Country'] == "HK"], globals()["HK_EQ"], on= ['Date', 'Country'], how='left')
df_6 = pd.merge(Equity_all[Equity_all['Country'] == "TH"], globals()["TH_EQ"], on= ['Date', 'Country'], how='left')
df_7 = pd.merge(Equity_all[Equity_all['Country'] == "GB"], globals()["GB_EQ"], on= ['Date', 'Country'], how='left')
df_8 = pd.merge(Equity_all[Equity_all['Country'] == "IN"], globals()["IN_EQ"], on= ['Date', 'Country'], how='left')
df_9 = pd.merge(Equity_all[Equity_all['Country'] == "QA"], QA_EQ, on= ['Date', 'Country'], how='left')
df_10 = pd.merge(Equity_all[Equity_all['Country'] == "SK"], globals()["SK_EQ"], on= ['Date', 'Country'], how='left')
df_11 = pd.merge(Equity_all[Equity_all['Country'] == "UAE"], globals()["UAE_EQ"], on= ['Date', 'Country'], how='left')

In [76]:
def calc_forex(df, x):
    if x == "no":
        df['Open'] = df['Open_x']*df['Open_y']
        df['High'] = df['High_x']*df['High_y']
        df['Low'] = df['Low_x']*df['Low_y']
        df['Close'] = df['Close_x']*df['Close_y']
        df = df[['Date', 'Open', 'High', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Bank', 'Country', 'foreign_dummy', 'private_dummy', 'public_dummy', 'small_finance_dummy']]
        return df
    elif x == "yes":
        df['Open'] = df['Open_x']*df['Open_y']
        df['High'] = df['High_x']*df['High_y']
        df['Low'] = df['Low_x']*df['Low_y']
        df['Close'] = df['Close_x']*df['Close_y']
        df = df[['Date', 'Open', 'High', 'Close', 'Volume_x', 'Dividends_x', 'Stock Splits_x', 'Bank', 'Country', 'foreign_dummy', 'private_dummy', 'public_dummy', 'small_finance_dummy']]
        df = df.rename(columns = {'Dividends_x':'Dividends', 'Stock Splits_x':'Stock Splits', 'Volume_x':'Volume'})
        return df

In [77]:
df_1 = calc_forex(df_1, "no")
df_2 = calc_forex(df_2, "yes")
df_3 = calc_forex(df_3, "yes")
df_4 = calc_forex(df_4, "yes")
df_5 = calc_forex(df_5, "yes")
df_6 = calc_forex(df_6, "yes")
df_7 = calc_forex(df_7, "yes")
df_8 = calc_forex(df_8, "yes")
df_9 = calc_forex(df_9, "no")
df_10 = calc_forex(df_10, "yes")
df_11 = calc_forex(df_11, "yes")

In [78]:
Equity_all = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11])

In [79]:
#Equity_all.to_csv(f'/Users/uddhav/Desktop/Thesis/bank_equity_csv/All_equity.csv')

In [80]:
Equity_excel = Equity_all[['Bank','Date','Open']]
Equity_excel = Equity_excel.pivot_table(index='Date', columns='Bank', values='Open', aggfunc='first')
Equity_excel

Bank,AMERICAN EXPRESS BANKING CORP.,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AXIS BANK LIMITED,BANDHAN BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",BANK OF BARODA,BANK OF CHINA LIMITED,BANK OF INDIA,BANK OF MAHARASHTRA,BANK OF NOVA SCOTIA,...,STANDARD CHARTERED BANK,STATE BANK OF INDIA,THE DHANALAKSHMI BANK LTD,UBS AG,UCO BANK,UJJIVAN SMALL FINANCE BANK LIMITED,UNION BANK OF INDIA,UNITED OVERSEAS BANK LTD,WESTPAC BANKING CORPORATION,YES BANK LTD.
Date,,,,,,,,,,,,,,,,,,,,,
2012-07-02,2747.640069,NaN,176.558408,NaN,374.656144,100.087361,NaN,306.279975,44.120469,1668.237975,...,88814.002327,143.669415,54.450001,482.621841,72.728198,NaN,180.431253,536.897394,673.223520,47.983770
2012-07-03,2759.802384,NaN,178.962122,NaN,367.318529,100.277996,10.324819,307.468138,44.924284,1678.134305,...,90363.307505,145.792934,55.000000,478.947086,75.283270,NaN,178.686111,535.525458,675.772998,48.723064
2012-07-04,NaN,NaN,181.227439,NaN,NaN,99.815013,10.197838,309.800459,44.879624,NaN,...,89979.219715,145.593831,54.900002,NaN,75.055132,NaN,180.218461,NaN,668.718561,49.539809
2012-07-05,2736.165702,NaN,179.844029,NaN,360.240410,99.603945,10.073842,307.468139,44.433064,1679.820692,...,89776.944469,147.322528,55.450001,461.057804,74.553249,NaN,180.346137,527.168122,666.580020,51.860590
2012-07-06,2727.020977,NaN,180.189889,NaN,347.358453,99.815009,9.925755,312.352769,45.102905,1681.159933,...,89577.870327,148.049190,62.500000,450.513578,75.739531,NaN,181.154852,530.791237,680.564680,53.473774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,8378.869209,4657.783381,741.870942,501.580677,2269.200823,96.814705,22.058869,65.211528,11.262448,3173.759888,...,62498.182740,326.358599,14.150000,768.678283,16.200001,55.119148,55.748333,1121.958773,991.704525,50.099998
2019-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-26,8395.710388,NaN,737.877017,495.046057,2281.613083,97.203129,NaN,64.559417,11.502075,3182.796727,...,NaN,326.652850,14.050000,769.417481,16.100000,53.855391,55.416784,1123.943974,NaN,51.000000


In [81]:
#Equity_excel.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_eq.xlsx')

In [82]:
for j, i in Private_Banks_eq.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2019-12-31')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Private_Banks_eq.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

Failed for CSB BANK LIMITED: Can only use .dt accessor with datetimelike values
Failed for IDFC FIRST BANK LIMITED: Can only use .dt accessor with datetimelike values
Failed for BANDHAN BANK LIMITED: Can only use .dt accessor with datetimelike values


/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2982817276.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2982817276.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2982817276.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2982817276.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959

In [83]:
keys_to_drop = ['CSB BANK LIMITED', 'IDFC FIRST BANK LIMITED', 'BANDHAN BANK LIMITED']
Private_Banks_eq2 = {key: value for key, value in Private_Banks_eq.items() if key not in keys_to_drop}

In [84]:
for j, i in Public_Banks_eq.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2019-12-31')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Public_Banks_eq.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2751082005.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2751082005.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2751082005.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2751082005.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959

In [85]:
for j, i in Foreign_Banks_eq.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2019-12-31')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Foreign_Banks_eq.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

Failed for AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED: Can only use .dt accessor with datetimelike values
Failed for EMIRATES NBD BANK (P.J.S.C.): Can only use .dt accessor with datetimelike values


/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2243786253.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2243786253.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2243786253.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2243786253.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959

In [86]:
keys_to_drop = ['AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED', 'EMIRATES NBD BANK (P.J.S.C.)']
Foreign_Banks_eq2 = {key: value for key, value in Foreign_Banks_eq.items() if key not in keys_to_drop}

In [87]:
for j, i in Small_Finance_eq.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2019-12-31')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Small_Finance_eq.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

Failed for UJJIVAN SMALL FINANCE BANK LIMITED: Can only use .dt accessor with datetimelike values


In [88]:
Private_mkt_merge = pd.concat([globals()[f'{key}_shares'] for key in Private_Banks_eq2.keys()], ignore_index=True)
Public_mkt_merge = pd.concat([globals()[f'{key}_shares'] for key in Public_Banks_eq.keys()], ignore_index=True)
Foreign_mkt_merge = pd.concat([globals()[f'{key}_shares'] for key in Foreign_Banks_eq2.keys()], ignore_index=True)
Shares_all = pd.concat([Private_mkt_merge, Public_mkt_merge, Foreign_mkt_merge])

In [89]:
Mkt = pd.merge(Equity_all, Shares_all, on = ['Date', 'Bank'])
Mkt['Mkt_cap'] = Mkt['Open'] * Mkt['share_count']
Mkt

,Date,Open,High,Close,Volume,Dividends,Stock Splits,Bank,Country,foreign_dummy,private_dummy,public_dummy,small_finance_dummy,share_count,Mkt_cap
0,2015-12-14,432.886295,436.942137,426.258453,974731,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378510000,1.029624e+12
1,2015-12-16,431.798168,437.139995,432.589539,524966,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378530000,1.027045e+12
2,2015-12-18,430.215411,436.051852,427.742340,738194,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378770000,1.023384e+12
3,2016-01-14,391.734416,395.196715,386.343109,1527567,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378950000,9.319166e+11
4,2016-01-19,372.790701,391.635489,388.865662,758663,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2379230000,8.869548e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8617,2019-11-07,562.982851,582.815756,580.437236,997816,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,577156992,3.249295e+11
8618,2019-11-07,562.982851,582.815756,580.437236,997816,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,581926976,3.276149e+11
8619,2019-11-08,580.109427,587.980704,576.869086,1083176,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,572425984,3.320697e+11
8620,2019-11-08,580.109427,587.980704,576.869086,1083176,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,577156992,3.348142e+11


In [90]:
Mkt_cap_excel = Mkt[['Bank','Date','Mkt_cap']]
Mkt_cap_excel = Mkt_cap_excel.pivot_table(index='Date', columns='Bank', values='Mkt_cap', aggfunc='first')
Mkt_cap_excel

Bank,AMERICAN EXPRESS BANKING CORP.,AXIS BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",BANK OF BARODA,BANK OF CHINA LIMITED,BANK OF INDIA,BANK OF MAHARASHTRA,BANK OF NOVA SCOTIA,BNP PARIBAS,CANARA BANK,...,SOUTH INDIAN BANK LTD,STANDARD CHARTERED BANK,STATE BANK OF INDIA,THE DHANALAKSHMI BANK LTD,UBS AG,UCO BANK,UNION BANK OF INDIA,UNITED OVERSEAS BANK LTD,WESTPAC BANKING CORPORATION,YES BANK LTD.
Date,,,,,,,,,,,,,,,,,,,,,
2012-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.117972e+09,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,NaN,NaN,NaN,NaN,7.410803e+12,NaN,6.735076e+10,3.883516e+12,NaN,2.350293e+11,...,NaN,NaN,NaN,NaN,2.826481e+12,1.440410e+11,1.993732e+11,1.959789e+12,NaN,1.337224e+11
2019-12-24,NaN,NaN,NaN,NaN,NaN,NaN,6.559374e+10,3.868147e+12,NaN,2.245044e+11,...,NaN,NaN,NaN,NaN,2.797904e+12,1.396975e+11,1.908165e+11,1.969329e+12,NaN,1.277785e+11
2019-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.962103e+12,NaN,NaN


In [91]:
#Mkt_cap_excel.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_mkt.xlsx')

In [92]:
asset_excel = balance_sheet[['Quarter', 'Banks', 'Total Assets_Domestic Operation']]
asset_excel = asset_excel.pivot_table(index='Quarter', columns='Banks', values='Total Assets_Domestic Operation', aggfunc='first')
asset_excel

Banks,AB BANK LIMITED,ABU DHABI COMMERCIAL BANK PJSC,ALLAHABAD BANK,AMERICAN EXPRESS BANKING CORP.,ANDHRA BANK,AU SMALL FINANCE BANK LIMITED,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AXIS BANK LIMITED,BANDHAN BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",...,UCO BANK,UJJIVAN SMALL FINANCE BANK LIMITED,UNION BANK OF INDIA,UNITED BANK OF INDIA,UNITED OVERSEAS BANK LTD,UTKARSH SMALL FINANCE BANK LIMITED,VIJAYA BANK,WESTPAC BANKING CORPORATION,WOORI BANK,YES BANK LTD.
Quarter,,,,,,,,,,,,,,,,,,,,,
2012Q2,1468900000,11792744000,1746491863000,21890861000,1291104700000,NaN,40552800000,2593429000000,NaN,140036458000,...,1645237600000,NaN,2549570400000,1035713028000,1619180000,NaN,987243169000,NaN,1789051000,782610300000
2012Q3,1622500000,12175766000,1811638407000,21833528000,1280588900000,NaN,42141700000,2756412700000,NaN,158401242000,...,1680350700000,NaN,2575470700000,1062146412000,1629574000,NaN,976985180000,NaN,2349334000,829924300000
2012Q4,1641200000,14185309000,1903282929000,21719223000,1343729100000,NaN,48483800000,2880939700000,NaN,223624345000,...,1755094400000,NaN,2730621200000,1074745242000,1629306000,NaN,1027025990000,NaN,2963761000,870352600000
2013Q1,1982700000,16405026000,1983993076000,23352962000,1464620200000,NaN,48995200000,3090510600000,NaN,184371476000,...,1817651100000,NaN,2994901900000,1146151131000,2043301000,NaN,1112008095000,8234046000,3836270000,991291600000
2013Q2,1678400000,16582670000,1991209940000,25334076000,1488657300000,NaN,49706400000,3007238700000,NaN,174956897000,...,1816441700000,NaN,3053791400000,1225145482000,1667577000,NaN,1192006388000,8295620000,4288568000,1008264700000
2013Q3,1708500000,18333079000,2009861231000,26243376000,1511272200000,NaN,53294200000,3181451000000,NaN,152206814000,...,1907334000000,NaN,3227535900000,1321260934000,2392460000,NaN,1205206777000,8432744000,7368951000,1022974800000
2013Q4,1677500000,21577513000,2091016606000,27061612000,1561690300000,NaN,51345700000,3181545100000,NaN,212319130000,...,2057641100000,NaN,3246293400000,1246125620000,3276597000,NaN,1272079864000,25852129000,6983668000,1014801000000
2014Q1,2074300000,22996308000,2131264514000,29796911000,1676235900000,NaN,56782700000,3430684200000,NaN,269299734000,...,2187907000000,NaN,3344574300000,1251049502000,3760690000,NaN,1376070072000,25853588000,6872511000,1091268700000
2014Q2,1979300000,22490430000,2054879009000,31479834000,1728172700000,NaN,45880800000,3407514900000,NaN,261687527000,...,2160236700000,NaN,3355415600000,1203620637000,3965823000,NaN,1305774642000,25177189000,7428567000,1098091200000


In [93]:
#asset_excel.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_asset.xlsx')

In [94]:
book_equity_excel = balance_sheet[['Quarter', 'Banks', 'Paid-up Share Capital_Domestic Operation']]
book_equity_excel = book_equity_excel.pivot_table(index='Quarter', columns='Banks', values='Paid-up Share Capital_Domestic Operation', aggfunc='first')
book_equity_excel

Banks,AB BANK LIMITED,ABU DHABI COMMERCIAL BANK PJSC,ALLAHABAD BANK,AMERICAN EXPRESS BANKING CORP.,ANDHRA BANK,AU SMALL FINANCE BANK LIMITED,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AXIS BANK LIMITED,BANDHAN BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",...,UCO BANK,UJJIVAN SMALL FINANCE BANK LIMITED,UNION BANK OF INDIA,UNITED BANK OF INDIA,UNITED OVERSEAS BANK LTD,UTKARSH SMALL FINANCE BANK LIMITED,VIJAYA BANK,WESTPAC BANKING CORPORATION,WOORI BANK,YES BANK LTD.
Quarter,,,,,,,,,,,,,,,,,,,,,
2012Q2,369800000,2170187000,5000262000,6177194000,5595800000,NaN,5854200000,4142900000,NaN,9853492000,...,24877100000,NaN,6615500000,11609989000,1549509000,NaN,16955391000,NaN,1605201000,3539600000
2012Q3,369800000,2170187000,5000262000,6177194000,5595800000,NaN,6397100000,4145300000,NaN,9853492000,...,24877100000,NaN,6615500000,11609989000,1549509000,NaN,16955391000,NaN,1605220000,3561100000
2012Q4,369800000,2170187000,5000262000,6177194000,5595800000,NaN,11311100000,4271600000,NaN,9853492000,...,24877100000,NaN,6615500000,11609989000,1549509000,NaN,16955391000,NaN,1605220000,3575600000
2013Q1,369800000,2170187000,5000262000,7527444000,5595800000,NaN,11311100000,4679500000,NaN,9853492000,...,25756300000,NaN,7077900000,11747069000,1549509000,NaN,16955391000,8087300000,1605220000,3586200000
2013Q2,369800000,2170187000,5000262000,7527444000,5595800000,NaN,11311100000,4688500000,NaN,9853492000,...,25756300000,NaN,7077900000,11747069000,1549509000,NaN,16955391000,8087300000,1605237000,3597200000
2013Q3,369800000,2170187000,5000262000,7527444000,5595800000,NaN,11311100000,4691900000,NaN,9853492000,...,25756300000,NaN,7077900000,11747069000,1549509000,NaN,16955391000,8087300000,4153261000,3604300000
2013Q4,369800000,2170187000,5446093000,7527444000,5896100000,NaN,11311100000,4692500000,NaN,9853492000,...,25756300000,NaN,7413100000,13547481000,1549509000,NaN,17544735000,8087300000,4153261000,3605600000
2014Q1,369800000,2170187000,5446093000,7527444000,5896100000,NaN,11311100000,4698400000,NaN,9853492000,...,10147100000,NaN,7413100000,13547481000,1549509000,NaN,8591194000,8087300000,4153261000,3606300000
2014Q2,369800000,2170187000,5446093000,7527444000,5896100000,NaN,11311100000,4713700000,NaN,9853492000,...,10147100000,NaN,7413100000,11573781000,1549509000,NaN,8591194000,8087300000,4153261000,4147100000


In [95]:
#book_equity_excel.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_book_eq.xlsx')

In [96]:
Private_Banks_eq2 = {'AXIS BANK LIMITED' : 'AXISBANK.BO', 'CITY UNION BANK LIMITED' : 'CUB.BO', 'CSB BANK LIMITED' : 'CSBBANK.BO', 'DCB BANK LIMITED' : 'DCBBANK.BO', 'FEDERAL BANK LTD' : 'FEDERALBNK.BO', 'HDFC BANK LTD.' : 'HDFCBANK.BO', 'ICICI BANK LIMITED':'ICICIBANK.BO', 'INDUSIND BANK LTD':'INDUSINDBK.BO', 'JAMMU & KASHMIR BANK LTD':'J&KBANK.BO', 'KARNATAKA BANK LTD':'KTKBANK.BO', 'KARUR VYSYA BANK LTD':'KARURVYSYA.BO', 'KOTAK MAHINDRA BANK LTD.':'KOTAKBANK.BO', 'RBL BANK LIMITED':'RBLBANK.BO', 'SOUTH INDIAN BANK LTD':'SOUTHBANK.BO', 'THE DHANALAKSHMI BANK LTD':'DHANBANK.BO', 'YES BANK LTD.':'YESBANK.BO', 'IDFC FIRST BANK LIMITED':'IDFCFIRSTB.BO', 'BANDHAN BANK LIMITED':'BANDHANBNK.BO'}
Public_Banks_eq2 = {'BANK OF BARODA':'BANKBARODA.BO', 'BANK OF INDIA':'BANKINDIA.BO', 'BANK OF MAHARASHTRA':'MAHABANK.BO', 'CANARA BANK':'CANBK.BO', 'CENTRAL BANK OF INDIA':'CENTRALBK.BO', 'IDBI BANK LIMITED':'IDBI.BO', 'INDIAN BANK':'INDIANB.BO', 'INDIAN OVERSEAS BANK':'IOB.BO', 'PUNJAB NATIONAL BANK':'PNB.BO', 'STATE BANK OF INDIA':'SBIN.BO', 'UCO BANK':'UCOBANK.BO', 'UNION BANK OF INDIA':'UNIONBANK.BO'}
Small_Finance_eq2 = {'UJJIVAN SMALL FINANCE BANK LIMITED':'UJJIVANSFB.BO', 'EQUITAS SMALL FINANCE BANK LIMITED':'EQUITASBNK.BO', 'SURYODAY SMALL FINANCE BANK LIMITED':'SURYODAY.BO'}
Foreign_Banks_eq2 = {'AMERICAN EXPRESS BANKING CORP.':'AXP', 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED':'AN3PH.AX', 'BANK OF AMERICA , NATIONAL ASSOCIATION':'BAC', 'BANK OF NOVA SCOTIA':'BNS', 'BNP PARIBAS':'BNP.PA', 'CITIBANK N.A':'C', 'COMMONWEALTH BANK OF AUSTRALIA':'CBA.AX', 'DEUTSCHE BANK AG':'DB', 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA':'1398.HK', 'JPMORGAN CHASE BANK NATIONAL ASSOCIATION':'JPM-PD', 'KRUNG THAI BANK PUBLIC COMPANY LIMITED':'KTB.BK', 'NATIONAL AUSTRALIA BANK':'NAB.AX', 'SOCIETE GENERALE':'GLE.PA', 'STANDARD CHARTERED BANK':'STAN.L', 'UBS AG':'UBS', 'UNITED OVERSEAS BANK LTD':'UOVEF', 'WESTPAC BANKING CORPORATION':'WBC.AX', 'PT BANK MAYBANK INDONESIA TBK':'BNII.JK', 'DOHA BANK Q.P.S.C':'DHBK.QA', 'INDUSTRIAL BANK OF KOREA':'024110.KS', 'Qatar National Bank (Q.P.S.C.)':'QNBK.QA', 'EMIRATES NBD BANK (P.J.S.C.)':'EMIRATESNBD.AE', 'BANK OF CHINA LIMITED':'3988.HK'}
Foreign_Banks_country2 = {'AMERICAN EXPRESS BANKING CORP.':'US', 'AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED':'AU', 'BANK OF AMERICA , NATIONAL ASSOCIATION':'US', 'BANK OF NOVA SCOTIA':'US', 'BNP PARIBAS':'EU', 'CITIBANK N.A':'US', 'COMMONWEALTH BANK OF AUSTRALIA':'AU', 'DEUTSCHE BANK AG':'US', 'INDUSTRIAL AND COMMERCIAL BANK OF CHINA':'HK', 'JPMORGAN CHASE BANK NATIONAL ASSOCIATION':'US', 'KRUNG THAI BANK PUBLIC COMPANY LIMITED':'TH', 'NATIONAL AUSTRALIA BANK':'AU', 'SOCIETE GENERALE':'EU', 'STANDARD CHARTERED BANK':'GB', 'UBS AG':'US', 'UNITED OVERSEAS BANK LTD':'US', 'WESTPAC BANKING CORPORATION':'AU', 'PT BANK MAYBANK INDONESIA TBK':'IN', 'DOHA BANK Q.P.S.C':'QA', 'INDUSTRIAL BANK OF KOREA':'SK', 'Qatar National Bank (Q.P.S.C.)':'QA', 'EMIRATES NBD BANK (P.J.S.C.)':'UAE', 'BANK OF CHINA LIMITED':'HK'}

In [97]:
Private_Banks_no_eq2 = ['ING VYSYA BANK LTD', 'LAKSHMI VILAS BANK LTD', 'NAINITAL BANK LTD', 'TAMILNAD MERCANTILE BANK LTD']
Public_Banks_no_eq2 = ['ALLAHABAD BANK', 'ANDHRA BANK', 'CORPORATION BANK', 'DENA BANK', 'ORIENTAL BANK OF COMMERCE', 'PUNJAB AND SIND BANK', 'STATE BANK OF BIKANER AND JAIPUR', 'STATE BANK OF MYSORE', 'STATE BANK OF PATIALA', 'STATE BANK OF TRAVANCORE', 'SYNDICATE BANK', 'UNITED BANK OF INDIA', 'VIJAYA BANK', 'BHARATIYA MAHILA BANK LTD.']
Small_Finance_no_eq2 = ['CAPITAL SMALL FINANCE BANK LIMITED', 'FINCARE SMALL FINANCE BANK LIMITED', 'NORTH EAST SMALL FINANCE BANK LIMITED', 'JANA SMALL FINANCE BANK LIMITED', 'ESAF SMALL FINANCE BANK LIMITED', 'UTKARSH SMALL FINANCE BANK LIMITED']
Foreign_Banks_no_eq2 = ['AB BANK LIMITED', 'ABU DHABI COMMERCIAL BANK PJSC', 'BANK OF BAHRAIN & KUWAIT B.S.C.', 'BANK OF CEYLON', 'BARCLAYS BANK PLC', 'COOPERATIEVE RABOBANK U.A.', 'CREDIT AGRICOLE CORPORATE AND INVESTMENT BANK', 'CREDIT SUISSE AG', 'CTBC BANK CO., LTD.', 'DBS BANK INDIA LTD.', 'FIRSTRAND BANK LTD', 'HONGKONG AND SHANGHAI BANKING CORPN.LTD.', 'HSBC BANK OMAN S.A.O.G.', 'JSC VTB BANK', 'KBC BANK NV', 'MASHREQ BANK PSC', 'MIZUHO BANK LTD', 'MUFG Bank Ltd', 'NatWest Markets Plc', 'SBERBANK', 'SBM BANK (INDIA) LTD.','SHINHAN BANK', 'SONALI BANK', 'WOORI BANK', 'SUMITOMO MITSUI BANKING CORPORATION', 'FIRST ABU DHABI BANK PJSC', 'KEB HANA BANK', 'KOOKMIN BANK']

In [98]:
for j, i in Private_Banks_eq2.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2023-09-30')

for i in Private_Banks_eq2.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    temp_df['Country'] = "IND"
    globals()[f'{i}_EQ'] = temp_df

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2451037422.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2451037422.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2451037422.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2451037422.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/245

In [99]:
for j, i in Public_Banks_eq2.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2023-09-30')

for i in Public_Banks_eq2.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    temp_df['Country'] = "IND"
    globals()[f'{i}_EQ'] = temp_df

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2015547946.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2015547946.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2015547946.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2015547946.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/201

In [100]:
for j, i in Small_Finance_eq2.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2023-09-30')

for i in Small_Finance_eq2.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    temp_df['Country'] = "IND"
    globals()[f'{i}_EQ'] = temp_df

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/653067162.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/653067162.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/653067162.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')


In [101]:
for j, i in Foreign_Banks_eq2.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2023-09-30')

for i in Foreign_Banks_eq2.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Bank'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    globals()[f'{i}_EQ'] = temp_df

for i,j in Foreign_Banks_country2.items():
    globals()[f'{i}_EQ']['Country'] = j

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/210531305.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/210531305.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/210531305.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/210531305.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2105313

In [102]:
s_p = yf.Ticker('BSE-500.BO')
s_p = s_p.history(start='2012-06-30', end='2023-09-30')
s_p['Bank'] = 'S&P500'
s_p['Country'] = 'IND'
s_p = s_p.reset_index()
s_p['Date'] = s_p['Date'].dt.to_period(freq='D')
s_p

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2384060013.py:6: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  s_p['Date'] = s_p['Date'].dt.to_period(freq='D')


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Bank,Country
0,2012-07-02,6695.990234,6724.910156,6677.089844,6700.359863,0,0.0,0.0,S&P500,IND
1,2012-07-03,6726.950195,6751.740234,6701.700195,6726.859863,0,0.0,0.0,S&P500,IND
2,2012-07-04,6752.970215,6758.959961,6716.209961,6751.189941,0,0.0,0.0,S&P500,IND
3,2012-07-05,6748.779785,6797.049805,6742.140137,6787.470215,0,0.0,0.0,S&P500,IND
4,2012-07-06,6786.490234,6786.490234,6743.430176,6766.750000,0,0.0,0.0,S&P500,IND
...,...,...,...,...,...,...,...,...,...,...
2743,2023-09-25,27381.839844,27427.470703,27261.740234,27373.480469,200,0.0,0.0,S&P500,IND
2744,2023-09-26,27402.949219,27438.109375,27338.490234,27374.019531,300,0.0,0.0,S&P500,IND
2745,2023-09-27,27397.339844,27495.509766,27284.970703,27479.019531,200,0.0,0.0,S&P500,IND
2746,2023-09-28,27596.400391,27596.400391,27179.980469,27213.529297,200,0.0,0.0,S&P500,IND


In [103]:
Private_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Private_Banks_eq2.keys()], ignore_index=True)
Private_eq_merge['foreign_dummy'] = 0
Private_eq_merge['private_dummy'] = 1
Private_eq_merge['public_dummy'] = 0
Private_eq_merge['small_finance_dummy'] = 0
Public_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Public_Banks_eq2.keys()], ignore_index=True)
Public_eq_merge['foreign_dummy'] = 0
Public_eq_merge['private_dummy'] = 0
Public_eq_merge['public_dummy'] = 1
Public_eq_merge['small_finance_dummy'] = 0
Small_Finance_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Small_Finance_eq2.keys()], ignore_index=True)
Small_Finance_eq_merge['foreign_dummy'] = 0
Small_Finance_eq_merge['private_dummy'] = 0
Small_Finance_eq_merge['public_dummy'] = 0
Small_Finance_eq_merge['small_finance_dummy'] = 1
Foreign_eq_merge = pd.concat([globals()[f'{key}_EQ'] for key in Foreign_Banks_eq2.keys()], ignore_index=True)
Foreign_eq_merge['foreign_dummy'] = 1
Foreign_eq_merge['private_dummy'] = 0
Foreign_eq_merge['public_dummy'] = 0
Foreign_eq_merge['small_finance_dummy'] = 0
Equity_all2 = pd.concat([Private_eq_merge, Public_eq_merge, Small_Finance_eq_merge, Foreign_eq_merge, s_p])

In [104]:
Equity_all2['Country'].unique()

array(['IND', 'US', 'AU', 'EU', 'HK', 'TH', 'GB', 'IN', 'QA', 'SK', 'UAE'],
      dtype=object)

In [105]:
Currency_ticks = {'US':'USDINR=X', 'AU':'AUDINR=X', 'EU':'EURINR=X', 'HK':'HKDINR=X', 'TH':'THBINR=X', 'GB':'GBPINR=X', 'IN':'IDRINR=X', 'QA':'QARUSD=X', 'SK':'KRWINR=X', 'UAE':'AEDINR=X'}
for j, i in Currency_ticks.items():
    df = yf.Ticker(i)
    globals()[f'{j}_EQ'] = df.history(start='2012-06-30', end='2023-09-30')

In [106]:
for i in Currency_ticks.keys():
    temp_df = globals()[f'{i}_EQ'].reset_index()
    temp_df['Country'] = i
    temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
    globals()[f'{i}_EQ'] = temp_df

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2068919482.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2068919482.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2068919482.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2068919482.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/206

In [107]:
QA_EQ = pd.merge(globals()["QA_EQ"], globals()["US_EQ"], on='Date')
QA_EQ['Open'] = QA_EQ['Open_x']*QA_EQ['Open_y']
QA_EQ['High'] = QA_EQ['High_x']*QA_EQ['High_y']
QA_EQ['Low'] = QA_EQ['Low_x']*QA_EQ['Low_y']
QA_EQ['Close'] = QA_EQ['Close_x']*QA_EQ['Close_y']
QA_EQ

,Date,Open_x,High_x,Low_x,Close_x,Volume_x,Dividends_x,Stock Splits_x,Country_x,Open_y,...,Low_y,Close_y,Volume_y,Dividends_y,Stock Splits_y,Country_y,Open,High,Low,Close
0,2012-07-02,0.284884,0.284884,0.274688,0.274801,0,0.0,0.0,QA,55.570000,...,55.349998,55.502998,0,0.0,0.0,US,15.831006,15.925018,15.203954,15.252267
1,2012-07-03,0.274846,0.274975,0.274657,0.274876,0,0.0,0.0,QA,55.428001,...,54.250000,55.428001,0,0.0,0.0,US,15.234169,15.241292,14.900162,15.235844
2,2012-07-04,0.274876,0.274937,0.274816,0.274876,0,0.0,0.0,QA,54.303001,...,54.169998,54.358002,0,0.0,0.0,US,14.926608,15.080281,14.886776,14.941726
3,2012-07-05,0.274891,0.274959,0.274657,0.274876,0,0.0,0.0,QA,54.563000,...,54.423000,54.478001,0,0.0,0.0,US,14.998900,15.183261,14.947678,14.974711
4,2012-07-06,0.274891,0.274944,0.274657,0.274876,0,0.0,0.0,QA,54.938000,...,54.938000,54.938000,0,0.0,0.0,US,15.101984,15.297353,15.089126,15.101154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2023-09-25,0.274801,0.274801,0.274801,0.274801,0,0.0,0.0,QA,82.974800,...,82.895699,82.974800,0,0.0,0.0,US,22.801540,22.855291,22.779802,22.801540
2926,2023-09-26,0.274763,0.274763,0.274763,0.274763,0,0.0,0.0,QA,83.028297,...,83.035301,83.028297,0,0.0,0.0,US,22.813106,22.915840,22.815030,22.813106
2927,2023-09-27,0.274801,0.274801,0.274801,0.274801,0,0.0,0.0,QA,83.298698,...,83.159302,83.298698,0,0.0,0.0,US,22.890547,22.895384,22.852241,22.890547
2928,2023-09-28,0.274763,0.274763,0.274763,0.274763,0,0.0,0.0,QA,83.222702,...,83.098503,83.222702,0,0.0,0.0,US,22.866521,22.900070,22.832395,22.866521


In [108]:
QA_EQ = QA_EQ[['Date','Open', 'High', 'Low', 'Close']]
QA_EQ['Country'] = "QA"
QA_EQ

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/3012904077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QA_EQ['Country'] = "QA"


,Date,Open,High,Low,Close,Country
0,2012-07-02,15.831006,15.925018,15.203954,15.252267,QA
1,2012-07-03,15.234169,15.241292,14.900162,15.235844,QA
2,2012-07-04,14.926608,15.080281,14.886776,14.941726,QA
3,2012-07-05,14.998900,15.183261,14.947678,14.974711,QA
4,2012-07-06,15.101984,15.297353,15.089126,15.101154,QA
...,...,...,...,...,...,...
2925,2023-09-25,22.801540,22.855291,22.779802,22.801540,QA
2926,2023-09-26,22.813106,22.915840,22.815030,22.813106,QA
2927,2023-09-27,22.890547,22.895384,22.852241,22.890547,QA
2928,2023-09-28,22.866521,22.900070,22.832395,22.866521,QA


In [109]:
INR_EQ = QA_EQ['Date']
INR_EQ = INR_EQ.to_frame()
INR_EQ['Open'] = 1
INR_EQ['High'] = 1
INR_EQ['Low'] = 1
INR_EQ['Close'] = 1
INR_EQ['Country'] = "IND"
INR_EQ

,Date,Open,High,Low,Close,Country
0,2012-07-02,1,1,1,1,IND
1,2012-07-03,1,1,1,1,IND
2,2012-07-04,1,1,1,1,IND
3,2012-07-05,1,1,1,1,IND
4,2012-07-06,1,1,1,1,IND
...,...,...,...,...,...,...
2925,2023-09-25,1,1,1,1,IND
2926,2023-09-26,1,1,1,1,IND
2927,2023-09-27,1,1,1,1,IND
2928,2023-09-28,1,1,1,1,IND


In [110]:
Equity_all2.set_index(['Date', 'Country'])
INR_EQ.set_index(['Date', 'Country'])
QA_EQ.set_index(['Date', 'Country'])

,,Open,High,Low,Close
Date,Country,,,,
2012-07-02,QA,15.831006,15.925018,15.203954,15.252267
2012-07-03,QA,15.234169,15.241292,14.900162,15.235844
2012-07-04,QA,14.926608,15.080281,14.886776,14.941726
2012-07-05,QA,14.998900,15.183261,14.947678,14.974711
2012-07-06,QA,15.101984,15.297353,15.089126,15.101154
...,...,...,...,...,...
2023-09-25,QA,22.801540,22.855291,22.779802,22.801540
2023-09-26,QA,22.813106,22.915840,22.815030,22.813106
2023-09-27,QA,22.890547,22.895384,22.852241,22.890547


In [111]:
df_1 = pd.merge(Equity_all2[Equity_all2['Country'] == "IND"], INR_EQ, on= ['Date', 'Country'], how='left')
df_2 = pd.merge(Equity_all2[Equity_all2['Country'] == "US"], globals()["US_EQ"], on= ['Date', 'Country'], how='left')
df_3 = pd.merge(Equity_all2[Equity_all2['Country'] == "AU"], globals()["AU_EQ"], on= ['Date', 'Country'], how='left')
df_4 = pd.merge(Equity_all2[Equity_all2['Country'] == "EU"], globals()["EU_EQ"], on= ['Date', 'Country'], how='left')
df_5 = pd.merge(Equity_all2[Equity_all2['Country'] == "HK"], globals()["HK_EQ"], on= ['Date', 'Country'], how='left')
df_6 = pd.merge(Equity_all2[Equity_all2['Country'] == "TH"], globals()["TH_EQ"], on= ['Date', 'Country'], how='left')
df_7 = pd.merge(Equity_all2[Equity_all2['Country'] == "GB"], globals()["GB_EQ"], on= ['Date', 'Country'], how='left')
df_8 = pd.merge(Equity_all2[Equity_all2['Country'] == "IN"], globals()["IN_EQ"], on= ['Date', 'Country'], how='left')
df_9 = pd.merge(Equity_all2[Equity_all2['Country'] == "QA"], QA_EQ, on= ['Date', 'Country'], how='left')
df_10 = pd.merge(Equity_all2[Equity_all2['Country'] == "SK"], globals()["SK_EQ"], on= ['Date', 'Country'], how='left')
df_11 = pd.merge(Equity_all2[Equity_all2['Country'] == "UAE"], globals()["UAE_EQ"], on= ['Date', 'Country'], how='left')

In [112]:
df_1 = calc_forex(df_1, "no")
df_2 = calc_forex(df_2, "yes")
df_3 = calc_forex(df_3, "yes")
df_4 = calc_forex(df_4, "yes")
df_5 = calc_forex(df_5, "yes")
df_6 = calc_forex(df_6, "yes")
df_7 = calc_forex(df_7, "yes")
df_8 = calc_forex(df_8, "yes")
df_9 = calc_forex(df_9, "no")
df_10 = calc_forex(df_10, "yes")
df_11 = calc_forex(df_11, "yes")

In [113]:
Equity_all_cov = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11])

In [114]:
#Equity_all_cov.to_csv(f'/Users/uddhav/Desktop/Thesis/bank_equity_csv/All_equity_cov.csv')

In [115]:
Equity_excel_cov = Equity_all_cov[['Bank','Date','Open']]
Equity_excel_cov = Equity_excel_cov.pivot_table(index='Date', columns='Bank', values='Open', aggfunc='first')
Equity_excel_cov

Bank,AMERICAN EXPRESS BANKING CORP.,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AXIS BANK LIMITED,BANDHAN BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",BANK OF BARODA,BANK OF CHINA LIMITED,BANK OF INDIA,BANK OF MAHARASHTRA,BANK OF NOVA SCOTIA,...,STATE BANK OF INDIA,SURYODAY SMALL FINANCE BANK LIMITED,THE DHANALAKSHMI BANK LTD,UBS AG,UCO BANK,UJJIVAN SMALL FINANCE BANK LIMITED,UNION BANK OF INDIA,UNITED OVERSEAS BANK LTD,WESTPAC BANKING CORPORATION,YES BANK LTD.
Date,,,,,,,,,,,,,,,,,,,,,
2012-07-02,2747.640699,NaN,176.558423,NaN,374.656117,100.087354,NaN,306.279975,44.120469,1668.238928,...,143.669384,NaN,54.450001,482.621841,72.728190,NaN,180.431253,536.897394,673.223411,47.983774
2012-07-03,2759.803846,NaN,178.962122,NaN,367.318609,100.277988,10.324818,307.468138,44.924284,1678.134618,...,145.792919,NaN,55.000000,478.947086,75.283270,NaN,178.686111,535.525458,675.773268,48.723056
2012-07-04,NaN,NaN,181.227455,NaN,NaN,99.815013,10.197837,309.800429,44.879624,NaN,...,145.593831,NaN,54.900002,NaN,75.055140,NaN,180.218431,NaN,668.718561,49.539813
2012-07-05,2736.166326,NaN,179.844044,NaN,360.240329,99.603945,10.073842,307.468139,44.433060,1679.821004,...,147.322513,NaN,55.450001,461.057804,74.553241,NaN,180.346122,527.168122,666.579862,51.860598
2012-07-06,2727.020349,NaN,180.189889,NaN,347.358427,99.815017,9.925755,312.352769,45.102909,1681.159401,...,148.049175,NaN,62.500000,450.513525,75.739524,NaN,181.154867,530.791237,680.564572,53.473767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,12537.411656,5366.351489,1012.049988,249.399994,2263.739217,216.050003,28.783749,110.580002,47.290001,3825.946284,...,600.049988,159.300003,28.809999,2094.283936,43.439999,49.840000,103.430000,1691.026357,1066.891865,17.719999
2023-09-26,12507.579853,5411.816528,1017.250000,253.000000,2252.019401,217.050003,28.867913,107.169998,46.340000,3768.798502,...,594.099976,159.949997,28.430000,2079.028644,42.750000,49.980000,102.989998,1762.690716,1083.272371,17.500000
2023-09-27,12527.641559,5400.313073,1015.799988,252.399994,2250.263296,213.699997,28.735155,108.330002,46.799999,3772.879161,...,594.299988,158.949997,28.250000,2018.327425,42.230000,50.200001,99.809998,1695.128545,1083.562170,17.379999


In [116]:
#Equity_excel_cov.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_eq_cov.xlsx')

In [117]:
for j, i in Private_Banks_eq2.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2023-09-30')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Private_Banks_eq2.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2639488109.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2639488109.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2639488109.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/2639488109.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959

In [118]:
keys_to_drop = ['CSB BANK LIMITED', 'IDFC FIRST BANK LIMITED', 'BANDHAN BANK LIMITED']
Private_Banks_eq3 = {key: value for key, value in Private_Banks_eq.items() if key not in keys_to_drop}

In [119]:
for j, i in Public_Banks_eq2.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2023-09-30')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Public_Banks_eq2.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/1239396700.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/1239396700.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/1239396700.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/1239396700.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959

In [120]:
for j, i in Foreign_Banks_eq2.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2023-09-30')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Foreign_Banks_eq2.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

Failed for AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED: Can only use .dt accessor with datetimelike values


/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/3524039706.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/3524039706.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/3524039706.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/3524039706.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959

In [121]:
keys_to_drop = ['AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED']
Foreign_Banks_eq3 = {key: value for key, value in Foreign_Banks_eq.items() if key not in keys_to_drop}

In [122]:
for j, i in Small_Finance_eq2.items():
    df = yf.Ticker(i)
    df_1 = df.get_shares_full(start='2012-06-30', end='2023-09-30')
    globals()[f'{j}_shares'] = pd.DataFrame(df_1)

for i in Small_Finance_eq2.keys():
    try:
        temp_df = globals()[f'{i}_shares'].reset_index()
        temp_df = temp_df.rename(columns={'index': 'Date', 0: 'share_count'})
        temp_df['Bank'] = i
        temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
        globals()[f'{i}_shares'] = temp_df
    except Exception as e:
        print(f"Failed for {i}: {e}")

/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/492726147.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/492726147.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')
/var/folders/5b/jc4253hj4rl3y0jlxy2y3trm0000gn/T/ipykernel_26959/492726147.py:11: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  temp_df['Date'] = temp_df['Date'].dt.to_period(freq='D')


In [123]:
Private_mkt_merge = pd.concat([globals()[f'{key}_shares'] for key in Private_Banks_eq3.keys()], ignore_index=True)
Public_mkt_merge = pd.concat([globals()[f'{key}_shares'] for key in Public_Banks_eq2.keys()], ignore_index=True)
Foreign_mkt_merge = pd.concat([globals()[f'{key}_shares'] for key in Foreign_Banks_eq3.keys()], ignore_index=True)
Small_finance_mkt_merge = pd.concat([globals()[f'{key}_shares'] for key in Small_Finance_eq2.keys()], ignore_index=True)
Shares_all2 = pd.concat([Private_mkt_merge, Public_mkt_merge, Foreign_mkt_merge, Small_finance_mkt_merge])

In [124]:
Mkt_cov = pd.merge(Equity_all_cov, Shares_all2, on = ['Date', 'Bank'])
Mkt_cov['Mkt_cap'] = Mkt_cov['Open'] * Mkt_cov['share_count']
Mkt_cov

,Date,Open,High,Close,Volume,Dividends,Stock Splits,Bank,Country,foreign_dummy,private_dummy,public_dummy,small_finance_dummy,share_count,Mkt_cap
0,2015-12-14,432.886326,436.942168,426.258484,974731,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378510000,1.029624e+12
1,2015-12-16,431.798138,437.139964,432.589508,524966,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378530000,1.027045e+12
2,2015-12-18,430.215380,436.051821,427.742310,738194,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378770000,1.023383e+12
3,2016-01-14,391.734416,395.196715,386.343109,1527567,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2378950000,9.319166e+11
4,2016-01-19,372.790701,391.635489,388.865662,758663,0.0,0.0,AXIS BANK LIMITED,IND,0.0,1.0,0.0,0.0,2379230000,8.869548e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16358,2023-07-06,648.807253,653.122990,640.716212,1391991,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,803644032,5.214101e+11
16359,2023-07-07,640.858168,648.537275,638.360379,995278,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,797425984,5.110370e+11
16360,2023-08-22,645.330381,652.761225,648.185267,445684,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,797425984,5.146032e+11
16361,2023-08-24,653.702396,659.688030,650.684992,390370,0.0,0.0,INDUSTRIAL BANK OF KOREA,SK,1.0,0.0,0.0,0.0,798185024,5.217755e+11


In [125]:
Mkt_cap_excel_cov = Mkt_cov[['Bank','Date','Mkt_cap']]
Mkt_cap_excel_cov = Mkt_cap_excel_cov.pivot_table(index='Date', columns='Bank', values='Mkt_cap', aggfunc='first')
Mkt_cap_excel_cov

Bank,AMERICAN EXPRESS BANKING CORP.,AXIS BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",BANK OF BARODA,BANK OF CHINA LIMITED,BANK OF INDIA,BANK OF MAHARASHTRA,BANK OF NOVA SCOTIA,BNP PARIBAS,CANARA BANK,...,STATE BANK OF INDIA,SURYODAY SMALL FINANCE BANK LIMITED,THE DHANALAKSHMI BANK LTD,UBS AG,UCO BANK,UJJIVAN SMALL FINANCE BANK LIMITED,UNION BANK OF INDIA,UNITED OVERSEAS BANK LTD,WESTPAC BANKING CORPORATION,YES BANK LTD.
Date,,,,,,,,,,,,,,,,,,,,,
2012-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.117972e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.807554e+12,NaN,NaN,NaN,2.829087e+12,NaN,NaN
2023-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.203237e+12,6.984273e+11,...,NaN,NaN,NaN,6.731853e+12,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-27,NaN,3.130452e+12,NaN,NaN,1.146751e+13,NaN,NaN,4.569862e+12,NaN,NaN,...,5.304787e+12,NaN,NaN,6.554700e+12,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
#Mkt_cap_excel_cov.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_mkt_cov.xlsx')

In [127]:
asset_excel_cov = balance_sheet_cov[['Quarter', 'Banks', 'Total Assets_Domestic Operation']]
asset_excel_cov = asset_excel_cov.pivot_table(index='Quarter', columns='Banks', values='Total Assets_Domestic Operation', aggfunc='first')
asset_excel_cov

Banks,AB BANK LIMITED,ABU DHABI COMMERCIAL BANK PJSC,ALLAHABAD BANK,AMERICAN EXPRESS BANKING CORP.,ANDHRA BANK,AU SMALL FINANCE BANK LIMITED,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AXIS BANK LIMITED,BANDHAN BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",...,UJJIVAN SMALL FINANCE BANK LIMITED,UNION BANK OF INDIA,UNITED BANK OF INDIA,UNITED OVERSEAS BANK LTD,UNITY SMALL FINANCE BANK LIMITED,UTKARSH SMALL FINANCE BANK LIMITED,VIJAYA BANK,WESTPAC BANKING CORPORATION,WOORI BANK,YES BANK LTD.
Quarter,,,,,,,,,,,,,,,,,,,,,
2012Q2,1468900000,11792744000,1746491863000,21890861000,1291104700000,NaN,40552800000,2593429000000,NaN,140036458000,...,NaN,2549570400000,1035713028000,1619180000,NaN,NaN,987243169000,NaN,1789051000,782610300000
2012Q3,1622500000,12175766000,1811638407000,21833528000,1280588900000,NaN,42141700000,2756412700000,NaN,158401242000,...,NaN,2575470700000,1062146412000,1629574000,NaN,NaN,976985180000,NaN,2349334000,829924300000
2012Q4,1641200000,14185309000,1903282929000,21719223000,1343729100000,NaN,48483800000,2880939700000,NaN,223624345000,...,NaN,2730621200000,1074745242000,1629306000,NaN,NaN,1027025990000,NaN,2963761000,870352600000
2013Q1,1982700000,16405026000,1983993076000,23352962000,1464620200000,NaN,48995200000,3090510600000,NaN,184371476000,...,NaN,2994901900000,1146151131000,2043301000,NaN,NaN,1112008095000,8234046000,3836270000,991291600000
2013Q2,1678400000,16582670000,1991209940000,25334076000,1488657300000,NaN,49706400000,3007238700000,NaN,174956897000,...,NaN,3053791400000,1225145482000,1667577000,NaN,NaN,1192006388000,8295620000,4288568000,1008264700000
2013Q3,1708500000,18333079000,2009861231000,26243376000,1511272200000,NaN,53294200000,3181451000000,NaN,152206814000,...,NaN,3227535900000,1321260934000,2392460000,NaN,NaN,1205206777000,8432744000,7368951000,1022974800000
2013Q4,1677500000,21577513000,2091016606000,27061612000,1561690300000,NaN,51345700000,3181545100000,NaN,212319130000,...,NaN,3246293400000,1246125620000,3276597000,NaN,NaN,1272079864000,25852129000,6983668000,1014801000000
2014Q1,2074300000,22996308000,2131264514000,29796911000,1676235900000,NaN,56782700000,3430684200000,NaN,269299734000,...,NaN,3344574300000,1251049502000,3760690000,NaN,NaN,1376070072000,25853588000,6872511000,1091268700000
2014Q2,1979300000,22490430000,2054879009000,31479834000,1728172700000,NaN,45880800000,3407514900000,NaN,261687527000,...,NaN,3355415600000,1203620637000,3965823000,NaN,NaN,1305774642000,25177189000,7428567000,1098091200000


In [128]:
#asset_excel_cov.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_asset_cov.xlsx')

In [129]:
book_equity_excel_cov = balance_sheet_cov[['Quarter', 'Banks', 'Paid-up Share Capital_Domestic Operation']]
book_equity_excel_cov = book_equity_excel_cov.pivot_table(index='Quarter', columns='Banks', values='Paid-up Share Capital_Domestic Operation', aggfunc='first')
book_equity_excel_cov

Banks,AB BANK LIMITED,ABU DHABI COMMERCIAL BANK PJSC,ALLAHABAD BANK,AMERICAN EXPRESS BANKING CORP.,ANDHRA BANK,AU SMALL FINANCE BANK LIMITED,AUSTRALIA AND NEW ZEALAND BANKING GROUP LIMITED,AXIS BANK LIMITED,BANDHAN BANK LIMITED,"BANK OF AMERICA , NATIONAL ASSOCIATION",...,UJJIVAN SMALL FINANCE BANK LIMITED,UNION BANK OF INDIA,UNITED BANK OF INDIA,UNITED OVERSEAS BANK LTD,UNITY SMALL FINANCE BANK LIMITED,UTKARSH SMALL FINANCE BANK LIMITED,VIJAYA BANK,WESTPAC BANKING CORPORATION,WOORI BANK,YES BANK LTD.
Quarter,,,,,,,,,,,,,,,,,,,,,
2012Q2,369800000,2170187000,5000262000,6177194000,5595800000,NaN,5854200000,4142900000,NaN,9853492000,...,NaN,6615500000,11609989000,1549509000,NaN,NaN,16955391000,NaN,1605201000,3539600000
2012Q3,369800000,2170187000,5000262000,6177194000,5595800000,NaN,6397100000,4145300000,NaN,9853492000,...,NaN,6615500000,11609989000,1549509000,NaN,NaN,16955391000,NaN,1605220000,3561100000
2012Q4,369800000,2170187000,5000262000,6177194000,5595800000,NaN,11311100000,4271600000,NaN,9853492000,...,NaN,6615500000,11609989000,1549509000,NaN,NaN,16955391000,NaN,1605220000,3575600000
2013Q1,369800000,2170187000,5000262000,7527444000,5595800000,NaN,11311100000,4679500000,NaN,9853492000,...,NaN,7077900000,11747069000,1549509000,NaN,NaN,16955391000,8087300000,1605220000,3586200000
2013Q2,369800000,2170187000,5000262000,7527444000,5595800000,NaN,11311100000,4688500000,NaN,9853492000,...,NaN,7077900000,11747069000,1549509000,NaN,NaN,16955391000,8087300000,1605237000,3597200000
2013Q3,369800000,2170187000,5000262000,7527444000,5595800000,NaN,11311100000,4691900000,NaN,9853492000,...,NaN,7077900000,11747069000,1549509000,NaN,NaN,16955391000,8087300000,4153261000,3604300000
2013Q4,369800000,2170187000,5446093000,7527444000,5896100000,NaN,11311100000,4692500000,NaN,9853492000,...,NaN,7413100000,13547481000,1549509000,NaN,NaN,17544735000,8087300000,4153261000,3605600000
2014Q1,369800000,2170187000,5446093000,7527444000,5896100000,NaN,11311100000,4698400000,NaN,9853492000,...,NaN,7413100000,13547481000,1549509000,NaN,NaN,8591194000,8087300000,4153261000,3606300000
2014Q2,369800000,2170187000,5446093000,7527444000,5896100000,NaN,11311100000,4713700000,NaN,9853492000,...,NaN,7413100000,11573781000,1549509000,NaN,NaN,8591194000,8087300000,4153261000,4147100000


In [130]:
#book_equity_excel_cov.to_excel('/Users/uddhav/Desktop/Thesis/sys_risk_book_eq_cov.xlsx')

In [131]:
def clean_state(df, x):
    if x == "agri":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,38,39])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([4])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "bank":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,38,39,40])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([4])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "const":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "ind":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "manu":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40,41])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3
    elif x == "serv":
        df_1 = df.iloc[~df.index.isin([0,1,2,3,4,39,40,41])]
        df_1.pop('Unnamed: 0')
        new_columns = [f"{df_1.iloc[0, i]}" for i in range(0,df_1.shape[1])]
        df_1.columns = new_columns
        col = df_1.columns
        col_2 = col[1:]
        col_3 = list(col_2)
        col_3.insert(0, 'state')
        df_1.columns = col_3
        df_2 = df_1.iloc[~df_1.index.isin([5])]
        df_2 = df_2.loc[:, ['state', '2014-15']]
        df_2.columns = ['state' , '2014-15_04', '2014-15_11']
        df_3 = df_2.loc[:, ['state', '2014-15_11']]
        df_3 = df_3.rename(columns={'2014-15_11':x})
        df_3[x] = df_3[x].replace('-', np.nan)
        df_3['state'] = df_3['state'].str.upper()
        return df_3

In [132]:
state_agri = clean_state(pd.read_excel('/Users/uddhav/Desktop/Thesis/state_data/state_agri.XLSX'), "agri")
state_banking = clean_state(pd.read_excel('/Users/uddhav/Desktop/Thesis/state_data/state_banking.XLSX'), "bank")
state_construction = clean_state(pd.read_excel('/Users/uddhav/Desktop/Thesis/state_data/state_construction.XLSX'), "const")
state_industry = clean_state(pd.read_excel('/Users/uddhav/Desktop/Thesis/state_data/state_industry.XLSX'), "ind")
state_manufacturing = clean_state(pd.read_excel('/Users/uddhav/Desktop/Thesis/state_data/state_manufacturing.XLSX'), "manu")
state_services = clean_state(pd.read_excel('/Users/uddhav/Desktop/Thesis/state_data/state_services.XLSX'), "serv")

In [133]:
state_vars = pd.merge(state_agri, state_banking, on = "state").merge(state_construction, on = "state").merge(state_industry, on = "state").merge(state_manufacturing, on = "state").merge(state_services, on = "state")

In [134]:
#state_vars.to_csv('/Users/uddhav/Desktop/Thesis/state_data/state_vars.csv')